In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from sklearn.ensemble import RandomForestRegressor
from ipywidgets import interact
import cartopy.crs as ccrs
import netCDF4 as nc

In [ ]:
#Creating a mask to be used for all our NA_temp region data

NA_mask_1deg = np.ones((44,74)) #initially masking every element

#function to convert from transcom coords to ERA coords (for my specific selection of coords in the data)
def coords_change_1(lat, lon):
    new_lat = (lat - 106)
    new_lon = (lon - 45)
    return(new_lat, new_lon)

#getting Transcom Region boundaries

filename_region = "CarbonTracker\\regions.nc"

h = nc.Dataset(filename_region)
transcom_regions = h.variables['transcom_regions'][:]

# -- Close file
h.close()

#unmasking the ERA datapoints which fall within the NA_temp region
for lati in range( 106,150 ):
    for longi in range( 45,119 ):
        if transcom_regions[lati][longi] == 2:
        
            m_lat,m_lon = coords_change_1(lati,longi)
            
            NA_mask_1deg[m_lat][m_lon] = 0
            
NA_mask_1deg_cyc = np.repeat(NA_mask_1deg[np.newaxis, :, :], 24, axis=0)



#Creating a mask to be used for all our NA_temp region data
# this is converting the mask from 1deg resolution to 0.1 deg resolution

NA_mask = np.ones((441,741)) #initially masking every element

#function to convert from transcom coords to ERA coords (for my specific selection of coords in the data)
def coords_change(lat, lon):
    new_lat = 10*(lat - 106)
    new_lon = 10*(lon - 45)
    return(new_lat, new_lon)

#downscaling function
def downmean(array, n):
    dim0 = array.shape[0]
    dim1 = array.shape[1]
    output = np.zeros(( dim0 // n , dim1 // n ))
    for lati in range( dim0 // n ):
        for longi in range( dim1 // n ):
            output[lati,longi] = np.mean(array[n*lati:n*(lati+1),n*longi:n*(longi+1)])
            
    return output

def downmean_3D(arr, n_0):
    n = 10*n_0
    
    dim0 = arr.shape[0]
    dim1 = arr.shape[1]
    dim2 = arr.shape[2]
    
    output = np.zeros((dim0, dim1//n, dim2//n))
    
    for k,layer in enumerate(arr[:]):
        output[k] = downmean(np.flip(layer,0),n)
    
    return output

def downmean_3D_mask(arr, n):
    
    dim0 = arr.shape[0]
    dim1 = arr.shape[1]
    dim2 = arr.shape[2]
    
    output = np.zeros((dim0, dim1//n, dim2//n))
    
    for k,layer in enumerate(arr[:]):
        output[k] = downmean(np.flip(layer,0),n)
    
    return output


In [ ]:
%%time
#data is shape (288, 441,741)
#first 0-23 in index[0] are the diurnal cycle for January of that 

filename_ERA = "mon_cycle_"

#doing 2000
h = nc.Dataset(filename_ERA + '2000.nc')

cyc_00_jan = {}
cyc_00_jan['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][0:24]),1), mask = NA_mask_1deg_cyc ) 
cyc_00_jan['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_00_jan['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_00_jan['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_00_jan['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_00_jan['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_00_jan['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_00_jan['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][0:24]),1), mask = NA_mask_1deg_cyc )


cyc_00_feb = {}
cyc_00_feb['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][24:48]),1), mask = NA_mask_1deg_cyc  )
cyc_00_feb['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_00_feb['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_00_feb['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_00_feb['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_00_feb['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_00_feb['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_00_feb['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][24:48]),1), mask = NA_mask_1deg_cyc )


cyc_00_mar = {}
cyc_00_mar['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][48:72]),1), mask = NA_mask_1deg_cyc ) 
cyc_00_mar['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_00_mar['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_00_mar['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_00_mar['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_00_mar['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_00_mar['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_00_mar['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][48:72]),1), mask = NA_mask_1deg_cyc )


cyc_00_apr = {}
cyc_00_apr['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][72:96]),1), mask = NA_mask_1deg_cyc ) 
cyc_00_apr['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_00_apr['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_00_apr['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_00_apr['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_00_apr['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_00_apr['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_00_apr['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][72:96]),1), mask = NA_mask_1deg_cyc )


cyc_00_may = {}
cyc_00_may['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][96:120]),1), mask = NA_mask_1deg_cyc ) 
cyc_00_may['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_00_may['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_00_may['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_00_may['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_00_may['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_00_may['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_00_may['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][96:120]),1), mask = NA_mask_1deg_cyc )


cyc_00_jun = {}
cyc_00_jun['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][120:144]),1), mask = NA_mask_1deg_cyc ) 
cyc_00_jun['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_00_jun['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_00_jun['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_00_jun['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_00_jun['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_00_jun['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_00_jun['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][120:144]),1), mask = NA_mask_1deg_cyc )


cyc_00_jul = {}
cyc_00_jul['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][144:168]),1), mask = NA_mask_1deg_cyc ) 
cyc_00_jul['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_00_jul['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_00_jul['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_00_jul['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_00_jul['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_00_jul['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_00_jul['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][144:168]),1), mask = NA_mask_1deg_cyc )


cyc_00_aug = {}
cyc_00_aug['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][168:192]),1), mask = NA_mask_1deg_cyc ) 
cyc_00_aug['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_00_aug['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_00_aug['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_00_aug['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_00_aug['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_00_aug['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_00_aug['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][168:192]),1), mask = NA_mask_1deg_cyc )


cyc_00_sep = {}
cyc_00_sep['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][192:216]),1), mask = NA_mask_1deg_cyc ) 
cyc_00_sep['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_00_sep['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_00_sep['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_00_sep['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_00_sep['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_00_sep['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_00_sep['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][192:216]),1), mask = NA_mask_1deg_cyc )


cyc_00_oct = {}
cyc_00_oct['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][216:240]),1), mask = NA_mask_1deg_cyc ) 
cyc_00_oct['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_00_oct['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_00_oct['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_00_oct['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_00_oct['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_00_oct['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_00_oct['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][216:240]),1), mask = NA_mask_1deg_cyc )


cyc_00_nov = {}
cyc_00_nov['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][240:264]),1), mask = NA_mask_1deg_cyc ) 
cyc_00_nov['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_00_nov['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_00_nov['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_00_nov['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_00_nov['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_00_nov['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_00_nov['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][240:264]),1), mask = NA_mask_1deg_cyc )


cyc_00_dec = {}
cyc_00_dec['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][264:288]),1), mask = NA_mask_1deg_cyc ) 
cyc_00_dec['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_00_dec['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_00_dec['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_00_dec['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_00_dec['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_00_dec['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_00_dec['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][264:288]),1), mask = NA_mask_1deg_cyc )


h.close()


#flattening the arrays for each month and saving them as dfs

flat_00_jan = {}
for name,arra in cyc_00_jan.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_00_jan[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_00_jan = pd.DataFrame(flat_00_jan)
df_00_jan['year'] = 2000
df_00_jan['month'] = 'jan'

flat_00_feb = {}
for name,arra in cyc_00_feb.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_00_feb[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_00_feb = pd.DataFrame(flat_00_feb)
df_00_feb['year'] = 2000
df_00_feb['month'] = 'feb'

flat_00_mar = {}
for name,arra in cyc_00_mar.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_00_mar[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_00_mar = pd.DataFrame(flat_00_mar)
df_00_mar['year'] = 2000
df_00_mar['month'] = 'mar'

flat_00_apr = {}
for name,arra in cyc_00_apr.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_00_apr[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_00_apr = pd.DataFrame(flat_00_apr)
df_00_apr['year'] = 2000
df_00_apr['month'] = 'apr'

flat_00_may = {}
for name,arra in cyc_00_may.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_00_may[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_00_may = pd.DataFrame(flat_00_may)
df_00_may['year'] = 2000
df_00_may['month'] = 'may'

flat_00_jun = {}
for name,arra in cyc_00_jun.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_00_jun[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_00_jun = pd.DataFrame(flat_00_jun)
df_00_jun['year'] = 2000
df_00_jun['month'] = 'jun'

flat_00_jul = {}
for name,arra in cyc_00_jul.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_00_jul[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_00_jul = pd.DataFrame(flat_00_jul)
df_00_jul['year'] = 2000
df_00_jul['month'] = 'jul'

flat_00_aug = {}
for name,arra in cyc_00_aug.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_00_aug[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_00_aug = pd.DataFrame(flat_00_aug)
df_00_aug['year'] = 2000
df_00_aug['month'] = 'aug'

flat_00_sep = {}
for name,arra in cyc_00_sep.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_00_sep[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_00_sep = pd.DataFrame(flat_00_sep)
df_00_sep['year'] = 2000
df_00_sep['month'] = 'sep'

flat_00_oct = {}
for name,arra in cyc_00_oct.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_00_oct[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_00_oct = pd.DataFrame(flat_00_oct)
df_00_oct['year'] = 2000
df_00_oct['month'] = 'oct'

flat_00_nov = {}
for name,arra in cyc_00_nov.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_00_nov[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_00_nov = pd.DataFrame(flat_00_nov)
df_00_nov['year'] = 2000
df_00_nov['month'] = 'nov'

flat_00_dec = {}
for name,arra in cyc_00_dec.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_00_dec[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_00_dec = pd.DataFrame(flat_00_dec)
df_00_dec['year'] = 2000
df_00_dec['month'] = 'dec'

df_00_jan.to_csv('\\df_00_jan.csv', index = False)
df_00_feb.to_csv('\\df_00_feb.csv', index = False)
df_00_mar.to_csv('\\df_00_mar.csv', index = False)
df_00_apr.to_csv('\\df_00_apr.csv', index = False)
df_00_may.to_csv('\\df_00_may.csv', index = False)
df_00_jun.to_csv('\\df_00_jun.csv', index = False)
df_00_jul.to_csv('\\df_00_jul.csv', index = False)
df_00_aug.to_csv('\\df_00_aug.csv', index = False)
df_00_sep.to_csv('\\df_00_sep.csv', index = False)
df_00_oct.to_csv('\\df_00_oct.csv', index = False)
df_00_nov.to_csv('\\df_00_nov.csv', index = False)
df_00_dec.to_csv('\\df_00_dec.csv', index = False)

In [ ]:
%%time

#2018 data

#data is shape (288, 441,741)
#first 0-23 in index[0] are the diurnal cycle for January of that 

filename_ERA = "mon_cycle_"

print('Doing Dictionaries')
#doing 2180
h = nc.Dataset(filename_ERA + '2018.nc')

cyc_18_jan = {}
cyc_18_jan['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][0:24]),1), mask = NA_mask_1deg_cyc ) 
cyc_18_jan['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_18_jan['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_18_jan['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_18_jan['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_18_jan['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_18_jan['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_18_jan['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][0:24]),1), mask = NA_mask_1deg_cyc )


cyc_18_feb = {}
cyc_18_feb['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][24:48]),1), mask = NA_mask_1deg_cyc  )
cyc_18_feb['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_18_feb['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_18_feb['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_18_feb['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_18_feb['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_18_feb['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_18_feb['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][24:48]),1), mask = NA_mask_1deg_cyc )


cyc_18_mar = {}
cyc_18_mar['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][48:72]),1), mask = NA_mask_1deg_cyc ) 
cyc_18_mar['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_18_mar['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_18_mar['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_18_mar['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_18_mar['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_18_mar['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_18_mar['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][48:72]),1), mask = NA_mask_1deg_cyc )


cyc_18_apr = {}
cyc_18_apr['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][72:96]),1), mask = NA_mask_1deg_cyc ) 
cyc_18_apr['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_18_apr['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_18_apr['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_18_apr['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_18_apr['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_18_apr['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_18_apr['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][72:96]),1), mask = NA_mask_1deg_cyc )


cyc_18_may = {}
cyc_18_may['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][96:120]),1), mask = NA_mask_1deg_cyc ) 
cyc_18_may['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_18_may['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_18_may['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_18_may['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_18_may['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_18_may['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_18_may['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][96:120]),1), mask = NA_mask_1deg_cyc )


cyc_18_jun = {}
cyc_18_jun['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][120:144]),1), mask = NA_mask_1deg_cyc ) 
cyc_18_jun['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_18_jun['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_18_jun['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_18_jun['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_18_jun['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_18_jun['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_18_jun['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][120:144]),1), mask = NA_mask_1deg_cyc )


cyc_18_jul = {}
cyc_18_jul['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][144:168]),1), mask = NA_mask_1deg_cyc ) 
cyc_18_jul['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_18_jul['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_18_jul['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_18_jul['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_18_jul['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_18_jul['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_18_jul['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][144:168]),1), mask = NA_mask_1deg_cyc )


cyc_18_aug = {}
cyc_18_aug['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][168:192]),1), mask = NA_mask_1deg_cyc ) 
cyc_18_aug['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_18_aug['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_18_aug['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_18_aug['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_18_aug['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_18_aug['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_18_aug['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][168:192]),1), mask = NA_mask_1deg_cyc )


cyc_18_sep = {}
cyc_18_sep['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][192:216]),1), mask = NA_mask_1deg_cyc ) 
cyc_18_sep['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_18_sep['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_18_sep['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_18_sep['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_18_sep['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_18_sep['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_18_sep['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][192:216]),1), mask = NA_mask_1deg_cyc )


cyc_18_oct = {}
cyc_18_oct['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][216:240]),1), mask = NA_mask_1deg_cyc ) 
cyc_18_oct['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_18_oct['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_18_oct['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_18_oct['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_18_oct['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_18_oct['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_18_oct['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][216:240]),1), mask = NA_mask_1deg_cyc )


cyc_18_nov = {}
cyc_18_nov['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][240:264]),1), mask = NA_mask_1deg_cyc ) 
cyc_18_nov['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_18_nov['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_18_nov['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_18_nov['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_18_nov['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_18_nov['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_18_nov['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][240:264]),1), mask = NA_mask_1deg_cyc )


cyc_18_dec = {}
cyc_18_dec['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][264:288]),1), mask = NA_mask_1deg_cyc ) 
cyc_18_dec['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_18_dec['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_18_dec['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_18_dec['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_18_dec['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_18_dec['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_18_dec['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][264:288]),1), mask = NA_mask_1deg_cyc )


h.close()


#flattening the arrays for each month and saving them as dfs
print('flattening arrays')

flat_18_jan = {}
for name,arra in cyc_18_jan.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_18_jan[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_18_jan = pd.DataFrame(flat_18_jan)
df_18_jan['year'] = 2018
df_18_jan['month'] = 'jan'

flat_18_feb = {}
for name,arra in cyc_18_feb.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_18_feb[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_18_feb = pd.DataFrame(flat_18_feb)
df_18_feb['year'] = 2018
df_18_feb['month'] = 'feb'

flat_18_mar = {}
for name,arra in cyc_18_mar.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_18_mar[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_18_mar = pd.DataFrame(flat_18_mar)
df_18_mar['year'] = 2018
df_18_mar['month'] = 'mar'

flat_18_apr = {}
for name,arra in cyc_18_apr.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_18_apr[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_18_apr = pd.DataFrame(flat_18_apr)
df_18_apr['year'] = 2018
df_18_apr['month'] = 'apr'

flat_18_may = {}
for name,arra in cyc_18_may.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_18_may[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_18_may = pd.DataFrame(flat_18_may)
df_18_may['year'] = 2018
df_18_may['month'] = 'may'

flat_18_jun = {}
for name,arra in cyc_18_jun.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_18_jun[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_18_jun = pd.DataFrame(flat_18_jun)
df_18_jun['year'] = 2018
df_18_jun['month'] = 'jun'

flat_18_jul = {}
for name,arra in cyc_18_jul.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_18_jul[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_18_jul = pd.DataFrame(flat_18_jul)
df_18_jul['year'] = 2018
df_18_jul['month'] = 'jul'

flat_18_aug = {}
for name,arra in cyc_18_aug.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_18_aug[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_18_aug = pd.DataFrame(flat_18_aug)
df_18_aug['year'] = 2018
df_18_aug['month'] = 'aug'

flat_18_sep = {}
for name,arra in cyc_18_sep.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_18_sep[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_18_sep = pd.DataFrame(flat_18_sep)
df_18_sep['year'] = 2018
df_18_sep['month'] = 'sep'

flat_18_oct = {}
for name,arra in cyc_18_oct.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_18_oct[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_18_oct = pd.DataFrame(flat_18_oct)
df_18_oct['year'] = 2018
df_18_oct['month'] = 'oct'

flat_18_nov = {}
for name,arra in cyc_18_nov.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_18_nov[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_18_nov = pd.DataFrame(flat_18_nov)
df_18_nov['year'] = 2018
df_18_nov['month'] = 'nov'

flat_18_dec = {}
for name,arra in cyc_18_dec.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_18_dec[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_18_dec = pd.DataFrame(flat_18_dec)
df_18_dec['year'] = 2018
df_18_dec['month'] = 'dec'

print('saving dataframes')
df_18_jan.to_csv('\\df_18_jan.csv', index = False)
df_18_feb.to_csv('\\df_18_feb.csv', index = False)
df_18_mar.to_csv('\\df_18_mar.csv', index = False)
df_18_apr.to_csv('\\df_18_apr.csv', index = False)
df_18_may.to_csv('\\df_18_may.csv', index = False)
df_18_jun.to_csv('\\df_18_jun.csv', index = False)
df_18_jul.to_csv('\\df_18_jul.csv', index = False)
df_18_aug.to_csv('\\df_18_aug.csv', index = False)
df_18_sep.to_csv('\\df_18_sep.csv', index = False)
df_18_oct.to_csv('\\df_18_oct.csv', index = False)
df_18_nov.to_csv('\\df_18_nov.csv', index = False)
df_18_dec.to_csv('\\df_18_dec.csv', index = False)

In [ ]:
%%time

#2017 data

#data is shape (288, 441,741)
#first 0-23 in index[0] are the diurnal cycle for January of that 

filename_ERA = "mon_cycle_"

print('Doing Dictionaries')
#doing 2170
h = nc.Dataset(filename_ERA + '2017.nc')

cyc_17_jan = {}
cyc_17_jan['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][0:24]),1), mask = NA_mask_1deg_cyc ) 
cyc_17_jan['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_17_jan['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_17_jan['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_17_jan['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_17_jan['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_17_jan['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_17_jan['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][0:24]),1), mask = NA_mask_1deg_cyc )


cyc_17_feb = {}
cyc_17_feb['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][24:48]),1), mask = NA_mask_1deg_cyc  )
cyc_17_feb['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_17_feb['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_17_feb['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_17_feb['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_17_feb['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_17_feb['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_17_feb['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][24:48]),1), mask = NA_mask_1deg_cyc )


cyc_17_mar = {}
cyc_17_mar['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][48:72]),1), mask = NA_mask_1deg_cyc ) 
cyc_17_mar['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_17_mar['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_17_mar['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_17_mar['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_17_mar['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_17_mar['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_17_mar['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][48:72]),1), mask = NA_mask_1deg_cyc )


cyc_17_apr = {}
cyc_17_apr['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][72:96]),1), mask = NA_mask_1deg_cyc ) 
cyc_17_apr['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_17_apr['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_17_apr['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_17_apr['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_17_apr['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_17_apr['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_17_apr['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][72:96]),1), mask = NA_mask_1deg_cyc )


cyc_17_may = {}
cyc_17_may['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][96:120]),1), mask = NA_mask_1deg_cyc ) 
cyc_17_may['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_17_may['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_17_may['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_17_may['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_17_may['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_17_may['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_17_may['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][96:120]),1), mask = NA_mask_1deg_cyc )


cyc_17_jun = {}
cyc_17_jun['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][120:144]),1), mask = NA_mask_1deg_cyc ) 
cyc_17_jun['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_17_jun['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_17_jun['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_17_jun['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_17_jun['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_17_jun['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_17_jun['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][120:144]),1), mask = NA_mask_1deg_cyc )


cyc_17_jul = {}
cyc_17_jul['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][144:168]),1), mask = NA_mask_1deg_cyc ) 
cyc_17_jul['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_17_jul['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_17_jul['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_17_jul['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_17_jul['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_17_jul['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_17_jul['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][144:168]),1), mask = NA_mask_1deg_cyc )


cyc_17_aug = {}
cyc_17_aug['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][168:192]),1), mask = NA_mask_1deg_cyc ) 
cyc_17_aug['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_17_aug['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_17_aug['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_17_aug['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_17_aug['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_17_aug['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_17_aug['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][168:192]),1), mask = NA_mask_1deg_cyc )


cyc_17_sep = {}
cyc_17_sep['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][192:216]),1), mask = NA_mask_1deg_cyc ) 
cyc_17_sep['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_17_sep['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_17_sep['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_17_sep['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_17_sep['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_17_sep['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_17_sep['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][192:216]),1), mask = NA_mask_1deg_cyc )


cyc_17_oct = {}
cyc_17_oct['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][216:240]),1), mask = NA_mask_1deg_cyc ) 
cyc_17_oct['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_17_oct['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_17_oct['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_17_oct['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_17_oct['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_17_oct['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_17_oct['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][216:240]),1), mask = NA_mask_1deg_cyc )


cyc_17_nov = {}
cyc_17_nov['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][240:264]),1), mask = NA_mask_1deg_cyc ) 
cyc_17_nov['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_17_nov['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_17_nov['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_17_nov['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_17_nov['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_17_nov['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_17_nov['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][240:264]),1), mask = NA_mask_1deg_cyc )


cyc_17_dec = {}
cyc_17_dec['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][264:288]),1), mask = NA_mask_1deg_cyc ) 
cyc_17_dec['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_17_dec['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_17_dec['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_17_dec['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_17_dec['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_17_dec['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_17_dec['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][264:288]),1), mask = NA_mask_1deg_cyc )


h.close()


#flattening the arrays for each month and saving them as dfs
print('flattening arrays')

flat_17_jan = {}
for name,arra in cyc_17_jan.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_17_jan[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_17_jan = pd.DataFrame(flat_17_jan)
df_17_jan['year'] = 2017
df_17_jan['month'] = 'jan'

flat_17_feb = {}
for name,arra in cyc_17_feb.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_17_feb[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_17_feb = pd.DataFrame(flat_17_feb)
df_17_feb['year'] = 2017
df_17_feb['month'] = 'feb'

flat_17_mar = {}
for name,arra in cyc_17_mar.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_17_mar[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_17_mar = pd.DataFrame(flat_17_mar)
df_17_mar['year'] = 2017
df_17_mar['month'] = 'mar'

flat_17_apr = {}
for name,arra in cyc_17_apr.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_17_apr[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_17_apr = pd.DataFrame(flat_17_apr)
df_17_apr['year'] = 2017
df_17_apr['month'] = 'apr'

flat_17_may = {}
for name,arra in cyc_17_may.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_17_may[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_17_may = pd.DataFrame(flat_17_may)
df_17_may['year'] = 2017
df_17_may['month'] = 'may'

flat_17_jun = {}
for name,arra in cyc_17_jun.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_17_jun[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_17_jun = pd.DataFrame(flat_17_jun)
df_17_jun['year'] = 2017
df_17_jun['month'] = 'jun'

flat_17_jul = {}
for name,arra in cyc_17_jul.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_17_jul[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_17_jul = pd.DataFrame(flat_17_jul)
df_17_jul['year'] = 2017
df_17_jul['month'] = 'jul'

flat_17_aug = {}
for name,arra in cyc_17_aug.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_17_aug[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_17_aug = pd.DataFrame(flat_17_aug)
df_17_aug['year'] = 2017
df_17_aug['month'] = 'aug'

flat_17_sep = {}
for name,arra in cyc_17_sep.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_17_sep[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_17_sep = pd.DataFrame(flat_17_sep)
df_17_sep['year'] = 2017
df_17_sep['month'] = 'sep'

flat_17_oct = {}
for name,arra in cyc_17_oct.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_17_oct[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_17_oct = pd.DataFrame(flat_17_oct)
df_17_oct['year'] = 2017
df_17_oct['month'] = 'oct'

flat_17_nov = {}
for name,arra in cyc_17_nov.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_17_nov[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_17_nov = pd.DataFrame(flat_17_nov)
df_17_nov['year'] = 2017
df_17_nov['month'] = 'nov'

flat_17_dec = {}
for name,arra in cyc_17_dec.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_17_dec[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_17_dec = pd.DataFrame(flat_17_dec)
df_17_dec['year'] = 2017
df_17_dec['month'] = 'dec'

print('saving dataframes')
df_17_jan.to_csv('\\df_17_jan.csv', index = False)
df_17_feb.to_csv('\\df_17_feb.csv', index = False)
df_17_mar.to_csv('\\df_17_mar.csv', index = False)
df_17_apr.to_csv('\\df_17_apr.csv', index = False)
df_17_may.to_csv('\\df_17_may.csv', index = False)
df_17_jun.to_csv('\\df_17_jun.csv', index = False)
df_17_jul.to_csv('\\df_17_jul.csv', index = False)
df_17_aug.to_csv('\\df_17_aug.csv', index = False)
df_17_sep.to_csv('\\df_17_sep.csv', index = False)
df_17_oct.to_csv('\\df_17_oct.csv', index = False)
df_17_nov.to_csv('\\df_17_nov.csv', index = False)
df_17_dec.to_csv('\\df_17_dec.csv', index = False)

In [ ]:
%%time

#2016 data

#data is shape (288, 441,741)
#first 0-23 in index[0] are the diurnal cycle for January of that 

filename_ERA = "mon_cycle_"

print('Doing Dictionaries')
#doing 2016
h = nc.Dataset(filename_ERA + '2016.nc')

cyc_16_jan = {}
cyc_16_jan['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][0:24]),1), mask = NA_mask_1deg_cyc ) 
cyc_16_jan['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_16_jan['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_16_jan['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_16_jan['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_16_jan['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_16_jan['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_16_jan['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][0:24]),1), mask = NA_mask_1deg_cyc )


cyc_16_feb = {}
cyc_16_feb['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][24:48]),1), mask = NA_mask_1deg_cyc  )
cyc_16_feb['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_16_feb['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_16_feb['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_16_feb['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_16_feb['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_16_feb['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_16_feb['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][24:48]),1), mask = NA_mask_1deg_cyc )


cyc_16_mar = {}
cyc_16_mar['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][48:72]),1), mask = NA_mask_1deg_cyc ) 
cyc_16_mar['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_16_mar['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_16_mar['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_16_mar['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_16_mar['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_16_mar['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_16_mar['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][48:72]),1), mask = NA_mask_1deg_cyc )


cyc_16_apr = {}
cyc_16_apr['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][72:96]),1), mask = NA_mask_1deg_cyc ) 
cyc_16_apr['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_16_apr['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_16_apr['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_16_apr['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_16_apr['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_16_apr['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_16_apr['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][72:96]),1), mask = NA_mask_1deg_cyc )


cyc_16_may = {}
cyc_16_may['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][96:120]),1), mask = NA_mask_1deg_cyc ) 
cyc_16_may['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_16_may['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_16_may['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_16_may['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_16_may['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_16_may['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_16_may['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][96:120]),1), mask = NA_mask_1deg_cyc )


cyc_16_jun = {}
cyc_16_jun['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][120:144]),1), mask = NA_mask_1deg_cyc ) 
cyc_16_jun['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_16_jun['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_16_jun['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_16_jun['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_16_jun['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_16_jun['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_16_jun['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][120:144]),1), mask = NA_mask_1deg_cyc )


cyc_16_jul = {}
cyc_16_jul['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][144:168]),1), mask = NA_mask_1deg_cyc ) 
cyc_16_jul['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_16_jul['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_16_jul['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_16_jul['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_16_jul['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_16_jul['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_16_jul['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][144:168]),1), mask = NA_mask_1deg_cyc )


cyc_16_aug = {}
cyc_16_aug['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][168:192]),1), mask = NA_mask_1deg_cyc ) 
cyc_16_aug['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_16_aug['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_16_aug['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_16_aug['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_16_aug['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_16_aug['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_16_aug['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][168:192]),1), mask = NA_mask_1deg_cyc )


cyc_16_sep = {}
cyc_16_sep['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][192:216]),1), mask = NA_mask_1deg_cyc ) 
cyc_16_sep['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_16_sep['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_16_sep['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_16_sep['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_16_sep['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_16_sep['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_16_sep['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][192:216]),1), mask = NA_mask_1deg_cyc )


cyc_16_oct = {}
cyc_16_oct['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][216:240]),1), mask = NA_mask_1deg_cyc ) 
cyc_16_oct['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_16_oct['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_16_oct['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_16_oct['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_16_oct['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_16_oct['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_16_oct['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][216:240]),1), mask = NA_mask_1deg_cyc )


cyc_16_nov = {}
cyc_16_nov['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][240:264]),1), mask = NA_mask_1deg_cyc ) 
cyc_16_nov['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_16_nov['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_16_nov['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_16_nov['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_16_nov['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_16_nov['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_16_nov['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][240:264]),1), mask = NA_mask_1deg_cyc )


cyc_16_dec = {}
cyc_16_dec['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][264:288]),1), mask = NA_mask_1deg_cyc ) 
cyc_16_dec['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_16_dec['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_16_dec['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_16_dec['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_16_dec['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_16_dec['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_16_dec['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][264:288]),1), mask = NA_mask_1deg_cyc )


h.close()


#flattening the arrays for each month and saving them as dfs
print('flattening arrays')

flat_16_jan = {}
for name,arra in cyc_16_jan.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_16_jan[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_16_jan = pd.DataFrame(flat_16_jan)
df_16_jan['year'] = 2016
df_16_jan['month'] = 'jan'

flat_16_feb = {}
for name,arra in cyc_16_feb.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_16_feb[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_16_feb = pd.DataFrame(flat_16_feb)
df_16_feb['year'] = 2016
df_16_feb['month'] = 'feb'

flat_16_mar = {}
for name,arra in cyc_16_mar.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_16_mar[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_16_mar = pd.DataFrame(flat_16_mar)
df_16_mar['year'] = 2016
df_16_mar['month'] = 'mar'

flat_16_apr = {}
for name,arra in cyc_16_apr.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_16_apr[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_16_apr = pd.DataFrame(flat_16_apr)
df_16_apr['year'] = 2016
df_16_apr['month'] = 'apr'

flat_16_may = {}
for name,arra in cyc_16_may.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_16_may[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_16_may = pd.DataFrame(flat_16_may)
df_16_may['year'] = 2016
df_16_may['month'] = 'may'

flat_16_jun = {}
for name,arra in cyc_16_jun.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_16_jun[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_16_jun = pd.DataFrame(flat_16_jun)
df_16_jun['year'] = 2016
df_16_jun['month'] = 'jun'

flat_16_jul = {}
for name,arra in cyc_16_jul.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_16_jul[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_16_jul = pd.DataFrame(flat_16_jul)
df_16_jul['year'] = 2016
df_16_jul['month'] = 'jul'

flat_16_aug = {}
for name,arra in cyc_16_aug.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_16_aug[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_16_aug = pd.DataFrame(flat_16_aug)
df_16_aug['year'] = 2016
df_16_aug['month'] = 'aug'

flat_16_sep = {}
for name,arra in cyc_16_sep.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_16_sep[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_16_sep = pd.DataFrame(flat_16_sep)
df_16_sep['year'] = 2016
df_16_sep['month'] = 'sep'

flat_16_oct = {}
for name,arra in cyc_16_oct.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_16_oct[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_16_oct = pd.DataFrame(flat_16_oct)
df_16_oct['year'] = 2016
df_16_oct['month'] = 'oct'

flat_16_nov = {}
for name,arra in cyc_16_nov.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_16_nov[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_16_nov = pd.DataFrame(flat_16_nov)
df_16_nov['year'] = 2016
df_16_nov['month'] = 'nov'

flat_16_dec = {}
for name,arra in cyc_16_dec.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_16_dec[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_16_dec = pd.DataFrame(flat_16_dec)
df_16_dec['year'] = 2016
df_16_dec['month'] = 'dec'

print('saving dataframes')
df_16_jan.to_csv('\\df_16_jan.csv', index = False)
df_16_feb.to_csv('\\df_16_feb.csv', index = False)
df_16_mar.to_csv('\\df_16_mar.csv', index = False)
df_16_apr.to_csv('\\df_16_apr.csv', index = False)
df_16_may.to_csv('\\df_16_may.csv', index = False)
df_16_jun.to_csv('\\df_16_jun.csv', index = False)
df_16_jul.to_csv('\\df_16_jul.csv', index = False)
df_16_aug.to_csv('\\df_16_aug.csv', index = False)
df_16_sep.to_csv('\\df_16_sep.csv', index = False)
df_16_oct.to_csv('\\df_16_oct.csv', index = False)
df_16_nov.to_csv('\\df_16_nov.csv', index = False)
df_16_dec.to_csv('\\df_16_dec.csv', index = False)

In [ ]:
%%time

#2015 data

#data is shape (288, 441,741)
#first 0-23 in index[0] are the diurnal cycle for January of that 

filename_ERA = "mon_cycle_"

print('Doing Dictionaries')
#doing 2015
h = nc.Dataset(filename_ERA + '2015.nc')

cyc_15_jan = {}
cyc_15_jan['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][0:24]),1), mask = NA_mask_1deg_cyc ) 
cyc_15_jan['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_15_jan['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_15_jan['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_15_jan['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_15_jan['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_15_jan['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_15_jan['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][0:24]),1), mask = NA_mask_1deg_cyc )


cyc_15_feb = {}
cyc_15_feb['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][24:48]),1), mask = NA_mask_1deg_cyc  )
cyc_15_feb['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_15_feb['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_15_feb['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_15_feb['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_15_feb['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_15_feb['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_15_feb['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][24:48]),1), mask = NA_mask_1deg_cyc )


cyc_15_mar = {}
cyc_15_mar['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][48:72]),1), mask = NA_mask_1deg_cyc ) 
cyc_15_mar['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_15_mar['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_15_mar['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_15_mar['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_15_mar['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_15_mar['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_15_mar['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][48:72]),1), mask = NA_mask_1deg_cyc )


cyc_15_apr = {}
cyc_15_apr['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][72:96]),1), mask = NA_mask_1deg_cyc ) 
cyc_15_apr['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_15_apr['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_15_apr['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_15_apr['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_15_apr['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_15_apr['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_15_apr['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][72:96]),1), mask = NA_mask_1deg_cyc )


cyc_15_may = {}
cyc_15_may['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][96:120]),1), mask = NA_mask_1deg_cyc ) 
cyc_15_may['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_15_may['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_15_may['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_15_may['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_15_may['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_15_may['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_15_may['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][96:120]),1), mask = NA_mask_1deg_cyc )


cyc_15_jun = {}
cyc_15_jun['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][120:144]),1), mask = NA_mask_1deg_cyc ) 
cyc_15_jun['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_15_jun['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_15_jun['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_15_jun['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_15_jun['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_15_jun['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_15_jun['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][120:144]),1), mask = NA_mask_1deg_cyc )


cyc_15_jul = {}
cyc_15_jul['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][144:168]),1), mask = NA_mask_1deg_cyc ) 
cyc_15_jul['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_15_jul['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_15_jul['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_15_jul['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_15_jul['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_15_jul['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_15_jul['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][144:168]),1), mask = NA_mask_1deg_cyc )


cyc_15_aug = {}
cyc_15_aug['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][168:192]),1), mask = NA_mask_1deg_cyc ) 
cyc_15_aug['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_15_aug['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_15_aug['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_15_aug['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_15_aug['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_15_aug['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_15_aug['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][168:192]),1), mask = NA_mask_1deg_cyc )


cyc_15_sep = {}
cyc_15_sep['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][192:216]),1), mask = NA_mask_1deg_cyc ) 
cyc_15_sep['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_15_sep['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_15_sep['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_15_sep['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_15_sep['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_15_sep['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_15_sep['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][192:216]),1), mask = NA_mask_1deg_cyc )


cyc_15_oct = {}
cyc_15_oct['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][216:240]),1), mask = NA_mask_1deg_cyc ) 
cyc_15_oct['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_15_oct['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_15_oct['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_15_oct['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_15_oct['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_15_oct['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_15_oct['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][216:240]),1), mask = NA_mask_1deg_cyc )


cyc_15_nov = {}
cyc_15_nov['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][240:264]),1), mask = NA_mask_1deg_cyc ) 
cyc_15_nov['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_15_nov['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_15_nov['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_15_nov['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_15_nov['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_15_nov['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_15_nov['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][240:264]),1), mask = NA_mask_1deg_cyc )


cyc_15_dec = {}
cyc_15_dec['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][264:288]),1), mask = NA_mask_1deg_cyc ) 
cyc_15_dec['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_15_dec['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_15_dec['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_15_dec['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_15_dec['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_15_dec['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_15_dec['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][264:288]),1), mask = NA_mask_1deg_cyc )


h.close()


#flattening the arrays for each month and saving them as dfs
print('flattening arrays')

flat_15_jan = {}
for name,arra in cyc_15_jan.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_15_jan[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_15_jan = pd.DataFrame(flat_15_jan)
df_15_jan['year'] = 2015
df_15_jan['month'] = 'jan'

flat_15_feb = {}
for name,arra in cyc_15_feb.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_15_feb[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_15_feb = pd.DataFrame(flat_15_feb)
df_15_feb['year'] = 2015
df_15_feb['month'] = 'feb'

flat_15_mar = {}
for name,arra in cyc_15_mar.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_15_mar[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_15_mar = pd.DataFrame(flat_15_mar)
df_15_mar['year'] = 2015
df_15_mar['month'] = 'mar'

flat_15_apr = {}
for name,arra in cyc_15_apr.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_15_apr[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_15_apr = pd.DataFrame(flat_15_apr)
df_15_apr['year'] = 2015
df_15_apr['month'] = 'apr'

flat_15_may = {}
for name,arra in cyc_15_may.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_15_may[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_15_may = pd.DataFrame(flat_15_may)
df_15_may['year'] = 2015
df_15_may['month'] = 'may'

flat_15_jun = {}
for name,arra in cyc_15_jun.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_15_jun[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_15_jun = pd.DataFrame(flat_15_jun)
df_15_jun['year'] = 2015
df_15_jun['month'] = 'jun'

flat_15_jul = {}
for name,arra in cyc_15_jul.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_15_jul[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_15_jul = pd.DataFrame(flat_15_jul)
df_15_jul['year'] = 2015
df_15_jul['month'] = 'jul'

flat_15_aug = {}
for name,arra in cyc_15_aug.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_15_aug[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_15_aug = pd.DataFrame(flat_15_aug)
df_15_aug['year'] = 2015
df_15_aug['month'] = 'aug'

flat_15_sep = {}
for name,arra in cyc_15_sep.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_15_sep[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_15_sep = pd.DataFrame(flat_15_sep)
df_15_sep['year'] = 2015
df_15_sep['month'] = 'sep'

flat_15_oct = {}
for name,arra in cyc_15_oct.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_15_oct[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_15_oct = pd.DataFrame(flat_15_oct)
df_15_oct['year'] = 2015
df_15_oct['month'] = 'oct'

flat_15_nov = {}
for name,arra in cyc_15_nov.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_15_nov[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_15_nov = pd.DataFrame(flat_15_nov)
df_15_nov['year'] = 2015
df_15_nov['month'] = 'nov'

flat_15_dec = {}
for name,arra in cyc_15_dec.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_15_dec[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_15_dec = pd.DataFrame(flat_15_dec)
df_15_dec['year'] = 2015
df_15_dec['month'] = 'dec'

print('saving dataframes')
df_15_jan.to_csv('\\df_15_jan.csv', index = False)
df_15_feb.to_csv('\\df_15_feb.csv', index = False)
df_15_mar.to_csv('\\df_15_mar.csv', index = False)
df_15_apr.to_csv('\\df_15_apr.csv', index = False)
df_15_may.to_csv('\\df_15_may.csv', index = False)
df_15_jun.to_csv('\\df_15_jun.csv', index = False)
df_15_jul.to_csv('\\df_15_jul.csv', index = False)
df_15_aug.to_csv('\\df_15_aug.csv', index = False)
df_15_sep.to_csv('\\df_15_sep.csv', index = False)
df_15_oct.to_csv('\\df_15_oct.csv', index = False)
df_15_nov.to_csv('\\df_15_nov.csv', index = False)
df_15_dec.to_csv('\\df_15_dec.csv', index = False)

In [ ]:
%%time

#2014 data

#data is shape (288, 441,741)
#first 0-23 in index[0] are the diurnal cycle for January of that 

filename_ERA = "mon_cycle_"

print('Doing Dictionaries')
#doing 2014
h = nc.Dataset(filename_ERA + '2014.nc')

cyc_14_jan = {}
cyc_14_jan['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][0:24]),1), mask = NA_mask_1deg_cyc ) 
cyc_14_jan['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_14_jan['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_14_jan['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_14_jan['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_14_jan['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_14_jan['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_14_jan['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][0:24]),1), mask = NA_mask_1deg_cyc )


cyc_14_feb = {}
cyc_14_feb['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][24:48]),1), mask = NA_mask_1deg_cyc  )
cyc_14_feb['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_14_feb['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_14_feb['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_14_feb['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_14_feb['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_14_feb['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_14_feb['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][24:48]),1), mask = NA_mask_1deg_cyc )


cyc_14_mar = {}
cyc_14_mar['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][48:72]),1), mask = NA_mask_1deg_cyc ) 
cyc_14_mar['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_14_mar['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_14_mar['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_14_mar['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_14_mar['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_14_mar['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_14_mar['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][48:72]),1), mask = NA_mask_1deg_cyc )


cyc_14_apr = {}
cyc_14_apr['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][72:96]),1), mask = NA_mask_1deg_cyc ) 
cyc_14_apr['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_14_apr['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_14_apr['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_14_apr['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_14_apr['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_14_apr['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_14_apr['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][72:96]),1), mask = NA_mask_1deg_cyc )


cyc_14_may = {}
cyc_14_may['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][96:120]),1), mask = NA_mask_1deg_cyc ) 
cyc_14_may['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_14_may['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_14_may['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_14_may['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_14_may['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_14_may['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_14_may['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][96:120]),1), mask = NA_mask_1deg_cyc )


cyc_14_jun = {}
cyc_14_jun['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][120:144]),1), mask = NA_mask_1deg_cyc ) 
cyc_14_jun['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_14_jun['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_14_jun['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_14_jun['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_14_jun['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_14_jun['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_14_jun['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][120:144]),1), mask = NA_mask_1deg_cyc )


cyc_14_jul = {}
cyc_14_jul['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][144:168]),1), mask = NA_mask_1deg_cyc ) 
cyc_14_jul['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_14_jul['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_14_jul['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_14_jul['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_14_jul['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_14_jul['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_14_jul['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][144:168]),1), mask = NA_mask_1deg_cyc )


cyc_14_aug = {}
cyc_14_aug['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][168:192]),1), mask = NA_mask_1deg_cyc ) 
cyc_14_aug['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_14_aug['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_14_aug['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_14_aug['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_14_aug['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_14_aug['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_14_aug['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][168:192]),1), mask = NA_mask_1deg_cyc )


cyc_14_sep = {}
cyc_14_sep['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][192:216]),1), mask = NA_mask_1deg_cyc ) 
cyc_14_sep['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_14_sep['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_14_sep['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_14_sep['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_14_sep['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_14_sep['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_14_sep['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][192:216]),1), mask = NA_mask_1deg_cyc )


cyc_14_oct = {}
cyc_14_oct['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][216:240]),1), mask = NA_mask_1deg_cyc ) 
cyc_14_oct['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_14_oct['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_14_oct['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_14_oct['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_14_oct['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_14_oct['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_14_oct['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][216:240]),1), mask = NA_mask_1deg_cyc )


cyc_14_nov = {}
cyc_14_nov['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][240:264]),1), mask = NA_mask_1deg_cyc ) 
cyc_14_nov['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_14_nov['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_14_nov['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_14_nov['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_14_nov['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_14_nov['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_14_nov['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][240:264]),1), mask = NA_mask_1deg_cyc )


cyc_14_dec = {}
cyc_14_dec['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][264:288]),1), mask = NA_mask_1deg_cyc ) 
cyc_14_dec['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_14_dec['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_14_dec['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_14_dec['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_14_dec['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_14_dec['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_14_dec['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][264:288]),1), mask = NA_mask_1deg_cyc )


h.close()


#flattening the arrays for each month and saving them as dfs
print('flattening arrays')

flat_14_jan = {}
for name,arra in cyc_14_jan.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_14_jan[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_14_jan = pd.DataFrame(flat_14_jan)
df_14_jan['year'] = 2014
df_14_jan['month'] = 'jan'

flat_14_feb = {}
for name,arra in cyc_14_feb.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_14_feb[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_14_feb = pd.DataFrame(flat_14_feb)
df_14_feb['year'] = 2014
df_14_feb['month'] = 'feb'

flat_14_mar = {}
for name,arra in cyc_14_mar.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_14_mar[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_14_mar = pd.DataFrame(flat_14_mar)
df_14_mar['year'] = 2014
df_14_mar['month'] = 'mar'

flat_14_apr = {}
for name,arra in cyc_14_apr.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_14_apr[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_14_apr = pd.DataFrame(flat_14_apr)
df_14_apr['year'] = 2014
df_14_apr['month'] = 'apr'

flat_14_may = {}
for name,arra in cyc_14_may.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_14_may[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_14_may = pd.DataFrame(flat_14_may)
df_14_may['year'] = 2014
df_14_may['month'] = 'may'

flat_14_jun = {}
for name,arra in cyc_14_jun.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_14_jun[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_14_jun = pd.DataFrame(flat_14_jun)
df_14_jun['year'] = 2014
df_14_jun['month'] = 'jun'

flat_14_jul = {}
for name,arra in cyc_14_jul.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_14_jul[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_14_jul = pd.DataFrame(flat_14_jul)
df_14_jul['year'] = 2014
df_14_jul['month'] = 'jul'

flat_14_aug = {}
for name,arra in cyc_14_aug.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_14_aug[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_14_aug = pd.DataFrame(flat_14_aug)
df_14_aug['year'] = 2014
df_14_aug['month'] = 'aug'

flat_14_sep = {}
for name,arra in cyc_14_sep.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_14_sep[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_14_sep = pd.DataFrame(flat_14_sep)
df_14_sep['year'] = 2014
df_14_sep['month'] = 'sep'

flat_14_oct = {}
for name,arra in cyc_14_oct.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_14_oct[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_14_oct = pd.DataFrame(flat_14_oct)
df_14_oct['year'] = 2014
df_14_oct['month'] = 'oct'

flat_14_nov = {}
for name,arra in cyc_14_nov.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_14_nov[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_14_nov = pd.DataFrame(flat_14_nov)
df_14_nov['year'] = 2014
df_14_nov['month'] = 'nov'

flat_14_dec = {}
for name,arra in cyc_14_dec.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_14_dec[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_14_dec = pd.DataFrame(flat_14_dec)
df_14_dec['year'] = 2014
df_14_dec['month'] = 'dec'

print('saving dataframes')
df_14_jan.to_csv('\\df_14_jan.csv', index = False)
df_14_feb.to_csv('\\df_14_feb.csv', index = False)
df_14_mar.to_csv('\\df_14_mar.csv', index = False)
df_14_apr.to_csv('\\df_14_apr.csv', index = False)
df_14_may.to_csv('\\df_14_may.csv', index = False)
df_14_jun.to_csv('\\df_14_jun.csv', index = False)
df_14_jul.to_csv('\\df_14_jul.csv', index = False)
df_14_aug.to_csv('\\df_14_aug.csv', index = False)
df_14_sep.to_csv('\\df_14_sep.csv', index = False)
df_14_oct.to_csv('\\df_14_oct.csv', index = False)
df_14_nov.to_csv('\\df_14_nov.csv', index = False)
df_14_dec.to_csv('\\df_14_dec.csv', index = False)

In [ ]:
%%time

#2013 data

#data is shape (288, 441,741)
#first 0-23 in index[0] are the diurnal cycle for January of that 

filename_ERA = "mon_cycle_"

print('Doing Dictionaries')
#doing 2013
h = nc.Dataset(filename_ERA + '2013.nc')

cyc_13_jan = {}
cyc_13_jan['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][0:24]),1), mask = NA_mask_1deg_cyc ) 
cyc_13_jan['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_13_jan['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_13_jan['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_13_jan['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_13_jan['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_13_jan['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_13_jan['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][0:24]),1), mask = NA_mask_1deg_cyc )


cyc_13_feb = {}
cyc_13_feb['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][24:48]),1), mask = NA_mask_1deg_cyc  )
cyc_13_feb['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_13_feb['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_13_feb['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_13_feb['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_13_feb['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_13_feb['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_13_feb['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][24:48]),1), mask = NA_mask_1deg_cyc )


cyc_13_mar = {}
cyc_13_mar['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][48:72]),1), mask = NA_mask_1deg_cyc ) 
cyc_13_mar['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_13_mar['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_13_mar['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_13_mar['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_13_mar['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_13_mar['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_13_mar['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][48:72]),1), mask = NA_mask_1deg_cyc )


cyc_13_apr = {}
cyc_13_apr['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][72:96]),1), mask = NA_mask_1deg_cyc ) 
cyc_13_apr['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_13_apr['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_13_apr['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_13_apr['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_13_apr['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_13_apr['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_13_apr['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][72:96]),1), mask = NA_mask_1deg_cyc )


cyc_13_may = {}
cyc_13_may['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][96:120]),1), mask = NA_mask_1deg_cyc ) 
cyc_13_may['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_13_may['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_13_may['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_13_may['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_13_may['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_13_may['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_13_may['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][96:120]),1), mask = NA_mask_1deg_cyc )


cyc_13_jun = {}
cyc_13_jun['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][120:144]),1), mask = NA_mask_1deg_cyc ) 
cyc_13_jun['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_13_jun['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_13_jun['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_13_jun['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_13_jun['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_13_jun['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_13_jun['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][120:144]),1), mask = NA_mask_1deg_cyc )


cyc_13_jul = {}
cyc_13_jul['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][144:168]),1), mask = NA_mask_1deg_cyc ) 
cyc_13_jul['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_13_jul['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_13_jul['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_13_jul['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_13_jul['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_13_jul['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_13_jul['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][144:168]),1), mask = NA_mask_1deg_cyc )


cyc_13_aug = {}
cyc_13_aug['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][168:192]),1), mask = NA_mask_1deg_cyc ) 
cyc_13_aug['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_13_aug['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_13_aug['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_13_aug['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_13_aug['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_13_aug['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_13_aug['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][168:192]),1), mask = NA_mask_1deg_cyc )


cyc_13_sep = {}
cyc_13_sep['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][192:216]),1), mask = NA_mask_1deg_cyc ) 
cyc_13_sep['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_13_sep['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_13_sep['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_13_sep['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_13_sep['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_13_sep['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_13_sep['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][192:216]),1), mask = NA_mask_1deg_cyc )


cyc_13_oct = {}
cyc_13_oct['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][216:240]),1), mask = NA_mask_1deg_cyc ) 
cyc_13_oct['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_13_oct['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_13_oct['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_13_oct['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_13_oct['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_13_oct['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_13_oct['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][216:240]),1), mask = NA_mask_1deg_cyc )


cyc_13_nov = {}
cyc_13_nov['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][240:264]),1), mask = NA_mask_1deg_cyc ) 
cyc_13_nov['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_13_nov['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_13_nov['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_13_nov['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_13_nov['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_13_nov['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_13_nov['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][240:264]),1), mask = NA_mask_1deg_cyc )


cyc_13_dec = {}
cyc_13_dec['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][264:288]),1), mask = NA_mask_1deg_cyc ) 
cyc_13_dec['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_13_dec['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_13_dec['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_13_dec['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_13_dec['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_13_dec['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_13_dec['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][264:288]),1), mask = NA_mask_1deg_cyc )


h.close()


#flattening the arrays for each month and saving them as dfs
print('flattening arrays')

flat_13_jan = {}
for name,arra in cyc_13_jan.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_13_jan[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_13_jan = pd.DataFrame(flat_13_jan)
df_13_jan['year'] = 2013
df_13_jan['month'] = 'jan'

flat_13_feb = {}
for name,arra in cyc_13_feb.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_13_feb[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_13_feb = pd.DataFrame(flat_13_feb)
df_13_feb['year'] = 2013
df_13_feb['month'] = 'feb'

flat_13_mar = {}
for name,arra in cyc_13_mar.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_13_mar[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_13_mar = pd.DataFrame(flat_13_mar)
df_13_mar['year'] = 2013
df_13_mar['month'] = 'mar'

flat_13_apr = {}
for name,arra in cyc_13_apr.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_13_apr[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_13_apr = pd.DataFrame(flat_13_apr)
df_13_apr['year'] = 2013
df_13_apr['month'] = 'apr'

flat_13_may = {}
for name,arra in cyc_13_may.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_13_may[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_13_may = pd.DataFrame(flat_13_may)
df_13_may['year'] = 2013
df_13_may['month'] = 'may'

flat_13_jun = {}
for name,arra in cyc_13_jun.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_13_jun[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_13_jun = pd.DataFrame(flat_13_jun)
df_13_jun['year'] = 2013
df_13_jun['month'] = 'jun'

flat_13_jul = {}
for name,arra in cyc_13_jul.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_13_jul[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_13_jul = pd.DataFrame(flat_13_jul)
df_13_jul['year'] = 2013
df_13_jul['month'] = 'jul'

flat_13_aug = {}
for name,arra in cyc_13_aug.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_13_aug[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_13_aug = pd.DataFrame(flat_13_aug)
df_13_aug['year'] = 2013
df_13_aug['month'] = 'aug'

flat_13_sep = {}
for name,arra in cyc_13_sep.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_13_sep[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_13_sep = pd.DataFrame(flat_13_sep)
df_13_sep['year'] = 2013
df_13_sep['month'] = 'sep'

flat_13_oct = {}
for name,arra in cyc_13_oct.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_13_oct[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_13_oct = pd.DataFrame(flat_13_oct)
df_13_oct['year'] = 2013
df_13_oct['month'] = 'oct'

flat_13_nov = {}
for name,arra in cyc_13_nov.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_13_nov[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_13_nov = pd.DataFrame(flat_13_nov)
df_13_nov['year'] = 2013
df_13_nov['month'] = 'nov'

flat_13_dec = {}
for name,arra in cyc_13_dec.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_13_dec[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_13_dec = pd.DataFrame(flat_13_dec)
df_13_dec['year'] = 2013
df_13_dec['month'] = 'dec'

print('saving dataframes')
df_13_jan.to_csv('\\df_13_jan.csv', index = False)
df_13_feb.to_csv('\\df_13_feb.csv', index = False)
df_13_mar.to_csv('\\df_13_mar.csv', index = False)
df_13_apr.to_csv('\\df_13_apr.csv', index = False)
df_13_may.to_csv('\\df_13_may.csv', index = False)
df_13_jun.to_csv('\\df_13_jun.csv', index = False)
df_13_jul.to_csv('\\df_13_jul.csv', index = False)
df_13_aug.to_csv('\\df_13_aug.csv', index = False)
df_13_sep.to_csv('\\df_13_sep.csv', index = False)
df_13_oct.to_csv('\\df_13_oct.csv', index = False)
df_13_nov.to_csv('\\df_13_nov.csv', index = False)
df_13_dec.to_csv('\\df_13_dec.csv', index = False)

In [ ]:
%%time

#2012 data

#data is shape (288, 441,741)
#first 0-23 in index[0] are the diurnal cycle for January of that 

filename_ERA = "mon_cycle_"

print('Doing Dictionaries')
#doing 2012
h = nc.Dataset(filename_ERA + '2012.nc')

cyc_12_jan = {}
cyc_12_jan['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][0:24]),1), mask = NA_mask_1deg_cyc ) 
cyc_12_jan['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_12_jan['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_12_jan['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_12_jan['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_12_jan['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_12_jan['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_12_jan['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][0:24]),1), mask = NA_mask_1deg_cyc )


cyc_12_feb = {}
cyc_12_feb['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][24:48]),1), mask = NA_mask_1deg_cyc  )
cyc_12_feb['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_12_feb['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_12_feb['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_12_feb['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_12_feb['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_12_feb['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_12_feb['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][24:48]),1), mask = NA_mask_1deg_cyc )


cyc_12_mar = {}
cyc_12_mar['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][48:72]),1), mask = NA_mask_1deg_cyc ) 
cyc_12_mar['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_12_mar['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_12_mar['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_12_mar['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_12_mar['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_12_mar['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_12_mar['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][48:72]),1), mask = NA_mask_1deg_cyc )


cyc_12_apr = {}
cyc_12_apr['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][72:96]),1), mask = NA_mask_1deg_cyc ) 
cyc_12_apr['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_12_apr['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_12_apr['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_12_apr['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_12_apr['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_12_apr['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_12_apr['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][72:96]),1), mask = NA_mask_1deg_cyc )


cyc_12_may = {}
cyc_12_may['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][96:120]),1), mask = NA_mask_1deg_cyc ) 
cyc_12_may['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_12_may['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_12_may['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_12_may['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_12_may['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_12_may['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_12_may['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][96:120]),1), mask = NA_mask_1deg_cyc )


cyc_12_jun = {}
cyc_12_jun['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][120:144]),1), mask = NA_mask_1deg_cyc ) 
cyc_12_jun['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_12_jun['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_12_jun['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_12_jun['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_12_jun['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_12_jun['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_12_jun['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][120:144]),1), mask = NA_mask_1deg_cyc )


cyc_12_jul = {}
cyc_12_jul['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][144:168]),1), mask = NA_mask_1deg_cyc ) 
cyc_12_jul['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_12_jul['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_12_jul['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_12_jul['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_12_jul['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_12_jul['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_12_jul['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][144:168]),1), mask = NA_mask_1deg_cyc )


cyc_12_aug = {}
cyc_12_aug['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][168:192]),1), mask = NA_mask_1deg_cyc ) 
cyc_12_aug['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_12_aug['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_12_aug['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_12_aug['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_12_aug['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_12_aug['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_12_aug['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][168:192]),1), mask = NA_mask_1deg_cyc )


cyc_12_sep = {}
cyc_12_sep['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][192:216]),1), mask = NA_mask_1deg_cyc ) 
cyc_12_sep['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_12_sep['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_12_sep['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_12_sep['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_12_sep['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_12_sep['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_12_sep['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][192:216]),1), mask = NA_mask_1deg_cyc )


cyc_12_oct = {}
cyc_12_oct['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][216:240]),1), mask = NA_mask_1deg_cyc ) 
cyc_12_oct['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_12_oct['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_12_oct['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_12_oct['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_12_oct['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_12_oct['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_12_oct['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][216:240]),1), mask = NA_mask_1deg_cyc )


cyc_12_nov = {}
cyc_12_nov['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][240:264]),1), mask = NA_mask_1deg_cyc ) 
cyc_12_nov['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_12_nov['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_12_nov['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_12_nov['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_12_nov['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_12_nov['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_12_nov['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][240:264]),1), mask = NA_mask_1deg_cyc )


cyc_12_dec = {}
cyc_12_dec['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][264:288]),1), mask = NA_mask_1deg_cyc ) 
cyc_12_dec['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_12_dec['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_12_dec['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_12_dec['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_12_dec['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_12_dec['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_12_dec['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][264:288]),1), mask = NA_mask_1deg_cyc )


h.close()


#flattening the arrays for each month and saving them as dfs
print('flattening arrays')

flat_12_jan = {}
for name,arra in cyc_12_jan.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_12_jan[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_12_jan = pd.DataFrame(flat_12_jan)
df_12_jan['year'] = 2012
df_12_jan['month'] = 'jan'

flat_12_feb = {}
for name,arra in cyc_12_feb.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_12_feb[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_12_feb = pd.DataFrame(flat_12_feb)
df_12_feb['year'] = 2012
df_12_feb['month'] = 'feb'

flat_12_mar = {}
for name,arra in cyc_12_mar.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_12_mar[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_12_mar = pd.DataFrame(flat_12_mar)
df_12_mar['year'] = 2012
df_12_mar['month'] = 'mar'

flat_12_apr = {}
for name,arra in cyc_12_apr.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_12_apr[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_12_apr = pd.DataFrame(flat_12_apr)
df_12_apr['year'] = 2012
df_12_apr['month'] = 'apr'

flat_12_may = {}
for name,arra in cyc_12_may.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_12_may[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_12_may = pd.DataFrame(flat_12_may)
df_12_may['year'] = 2012
df_12_may['month'] = 'may'

flat_12_jun = {}
for name,arra in cyc_12_jun.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_12_jun[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_12_jun = pd.DataFrame(flat_12_jun)
df_12_jun['year'] = 2012
df_12_jun['month'] = 'jun'

flat_12_jul = {}
for name,arra in cyc_12_jul.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_12_jul[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_12_jul = pd.DataFrame(flat_12_jul)
df_12_jul['year'] = 2012
df_12_jul['month'] = 'jul'

flat_12_aug = {}
for name,arra in cyc_12_aug.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_12_aug[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_12_aug = pd.DataFrame(flat_12_aug)
df_12_aug['year'] = 2012
df_12_aug['month'] = 'aug'

flat_12_sep = {}
for name,arra in cyc_12_sep.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_12_sep[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_12_sep = pd.DataFrame(flat_12_sep)
df_12_sep['year'] = 2012
df_12_sep['month'] = 'sep'

flat_12_oct = {}
for name,arra in cyc_12_oct.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_12_oct[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_12_oct = pd.DataFrame(flat_12_oct)
df_12_oct['year'] = 2012
df_12_oct['month'] = 'oct'

flat_12_nov = {}
for name,arra in cyc_12_nov.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_12_nov[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_12_nov = pd.DataFrame(flat_12_nov)
df_12_nov['year'] = 2012
df_12_nov['month'] = 'nov'

flat_12_dec = {}
for name,arra in cyc_12_dec.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_12_dec[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_12_dec = pd.DataFrame(flat_12_dec)
df_12_dec['year'] = 2012
df_12_dec['month'] = 'dec'

print('saving dataframes')
df_12_jan.to_csv('\\df_12_jan.csv', index = False)
df_12_feb.to_csv('\\df_12_feb.csv', index = False)
df_12_mar.to_csv('\\df_12_mar.csv', index = False)
df_12_apr.to_csv('\\df_12_apr.csv', index = False)
df_12_may.to_csv('\\df_12_may.csv', index = False)
df_12_jun.to_csv('\\df_12_jun.csv', index = False)
df_12_jul.to_csv('\\df_12_jul.csv', index = False)
df_12_aug.to_csv('\\df_12_aug.csv', index = False)
df_12_sep.to_csv('\\df_12_sep.csv', index = False)
df_12_oct.to_csv('\\df_12_oct.csv', index = False)
df_12_nov.to_csv('\\df_12_nov.csv', index = False)
df_12_dec.to_csv('\\df_12_dec.csv', index = False)

In [ ]:
%%time

#2011 data

#data is shape (288, 441,741)
#first 0-23 in index[0] are the diurnal cycle for January of that 

filename_ERA = "mon_cycle_"

print('Doing Dictionaries')
#doing 2011
h = nc.Dataset(filename_ERA + '2011.nc')

cyc_11_jan = {}
cyc_11_jan['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][0:24]),1), mask = NA_mask_1deg_cyc ) 
cyc_11_jan['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_11_jan['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_11_jan['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_11_jan['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_11_jan['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_11_jan['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_11_jan['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][0:24]),1), mask = NA_mask_1deg_cyc )


cyc_11_feb = {}
cyc_11_feb['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][24:48]),1), mask = NA_mask_1deg_cyc  )
cyc_11_feb['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_11_feb['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_11_feb['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_11_feb['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_11_feb['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_11_feb['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_11_feb['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][24:48]),1), mask = NA_mask_1deg_cyc )


cyc_11_mar = {}
cyc_11_mar['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][48:72]),1), mask = NA_mask_1deg_cyc ) 
cyc_11_mar['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_11_mar['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_11_mar['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_11_mar['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_11_mar['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_11_mar['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_11_mar['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][48:72]),1), mask = NA_mask_1deg_cyc )


cyc_11_apr = {}
cyc_11_apr['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][72:96]),1), mask = NA_mask_1deg_cyc ) 
cyc_11_apr['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_11_apr['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_11_apr['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_11_apr['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_11_apr['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_11_apr['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_11_apr['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][72:96]),1), mask = NA_mask_1deg_cyc )


cyc_11_may = {}
cyc_11_may['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][96:120]),1), mask = NA_mask_1deg_cyc ) 
cyc_11_may['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_11_may['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_11_may['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_11_may['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_11_may['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_11_may['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_11_may['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][96:120]),1), mask = NA_mask_1deg_cyc )


cyc_11_jun = {}
cyc_11_jun['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][120:144]),1), mask = NA_mask_1deg_cyc ) 
cyc_11_jun['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_11_jun['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_11_jun['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_11_jun['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_11_jun['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_11_jun['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_11_jun['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][120:144]),1), mask = NA_mask_1deg_cyc )


cyc_11_jul = {}
cyc_11_jul['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][144:168]),1), mask = NA_mask_1deg_cyc ) 
cyc_11_jul['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_11_jul['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_11_jul['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_11_jul['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_11_jul['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_11_jul['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_11_jul['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][144:168]),1), mask = NA_mask_1deg_cyc )


cyc_11_aug = {}
cyc_11_aug['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][168:192]),1), mask = NA_mask_1deg_cyc ) 
cyc_11_aug['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_11_aug['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_11_aug['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_11_aug['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_11_aug['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_11_aug['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_11_aug['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][168:192]),1), mask = NA_mask_1deg_cyc )


cyc_11_sep = {}
cyc_11_sep['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][192:216]),1), mask = NA_mask_1deg_cyc ) 
cyc_11_sep['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_11_sep['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_11_sep['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_11_sep['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_11_sep['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_11_sep['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_11_sep['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][192:216]),1), mask = NA_mask_1deg_cyc )


cyc_11_oct = {}
cyc_11_oct['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][216:240]),1), mask = NA_mask_1deg_cyc ) 
cyc_11_oct['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_11_oct['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_11_oct['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_11_oct['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_11_oct['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_11_oct['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_11_oct['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][216:240]),1), mask = NA_mask_1deg_cyc )


cyc_11_nov = {}
cyc_11_nov['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][240:264]),1), mask = NA_mask_1deg_cyc ) 
cyc_11_nov['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_11_nov['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_11_nov['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_11_nov['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_11_nov['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_11_nov['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_11_nov['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][240:264]),1), mask = NA_mask_1deg_cyc )


cyc_11_dec = {}
cyc_11_dec['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][264:288]),1), mask = NA_mask_1deg_cyc ) 
cyc_11_dec['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_11_dec['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_11_dec['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_11_dec['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_11_dec['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_11_dec['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_11_dec['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][264:288]),1), mask = NA_mask_1deg_cyc )


h.close()


#flattening the arrays for each month and saving them as dfs
print('flattening arrays')

flat_11_jan = {}
for name,arra in cyc_11_jan.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_11_jan[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_11_jan = pd.DataFrame(flat_11_jan)
df_11_jan['year'] = 2011
df_11_jan['month'] = 'jan'

flat_11_feb = {}
for name,arra in cyc_11_feb.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_11_feb[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_11_feb = pd.DataFrame(flat_11_feb)
df_11_feb['year'] = 2011
df_11_feb['month'] = 'feb'

flat_11_mar = {}
for name,arra in cyc_11_mar.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_11_mar[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_11_mar = pd.DataFrame(flat_11_mar)
df_11_mar['year'] = 2011
df_11_mar['month'] = 'mar'

flat_11_apr = {}
for name,arra in cyc_11_apr.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_11_apr[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_11_apr = pd.DataFrame(flat_11_apr)
df_11_apr['year'] = 2011
df_11_apr['month'] = 'apr'

flat_11_may = {}
for name,arra in cyc_11_may.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_11_may[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_11_may = pd.DataFrame(flat_11_may)
df_11_may['year'] = 2011
df_11_may['month'] = 'may'

flat_11_jun = {}
for name,arra in cyc_11_jun.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_11_jun[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_11_jun = pd.DataFrame(flat_11_jun)
df_11_jun['year'] = 2011
df_11_jun['month'] = 'jun'

flat_11_jul = {}
for name,arra in cyc_11_jul.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_11_jul[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_11_jul = pd.DataFrame(flat_11_jul)
df_11_jul['year'] = 2011
df_11_jul['month'] = 'jul'

flat_11_aug = {}
for name,arra in cyc_11_aug.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_11_aug[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_11_aug = pd.DataFrame(flat_11_aug)
df_11_aug['year'] = 2011
df_11_aug['month'] = 'aug'

flat_11_sep = {}
for name,arra in cyc_11_sep.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_11_sep[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_11_sep = pd.DataFrame(flat_11_sep)
df_11_sep['year'] = 2011
df_11_sep['month'] = 'sep'

flat_11_oct = {}
for name,arra in cyc_11_oct.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_11_oct[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_11_oct = pd.DataFrame(flat_11_oct)
df_11_oct['year'] = 2011
df_11_oct['month'] = 'oct'

flat_11_nov = {}
for name,arra in cyc_11_nov.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_11_nov[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_11_nov = pd.DataFrame(flat_11_nov)
df_11_nov['year'] = 2011
df_11_nov['month'] = 'nov'

flat_11_dec = {}
for name,arra in cyc_11_dec.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_11_dec[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_11_dec = pd.DataFrame(flat_11_dec)
df_11_dec['year'] = 2011
df_11_dec['month'] = 'dec'

print('saving dataframes')
df_11_jan.to_csv('\\df_11_jan.csv', index = False)
df_11_feb.to_csv('\\df_11_feb.csv', index = False)
df_11_mar.to_csv('\\df_11_mar.csv', index = False)
df_11_apr.to_csv('\\df_11_apr.csv', index = False)
df_11_may.to_csv('\\df_11_may.csv', index = False)
df_11_jun.to_csv('\\df_11_jun.csv', index = False)
df_11_jul.to_csv('\\df_11_jul.csv', index = False)
df_11_aug.to_csv('\\df_11_aug.csv', index = False)
df_11_sep.to_csv('\\df_11_sep.csv', index = False)
df_11_oct.to_csv('\\df_11_oct.csv', index = False)
df_11_nov.to_csv('\\df_11_nov.csv', index = False)
df_11_dec.to_csv('\\df_11_dec.csv', index = False)

In [ ]:
%%time

#2010 data

#data is shape (288, 441,741)
#first 0-23 in index[0] are the diurnal cycle for January of that 

filename_ERA = "mon_cycle_"

print('Doing Dictionaries')
#doing 2010
h = nc.Dataset(filename_ERA + '2010.nc')

cyc_10_jan = {}
cyc_10_jan['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][0:24]),1), mask = NA_mask_1deg_cyc ) 
cyc_10_jan['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_10_jan['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_10_jan['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_10_jan['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_10_jan['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_10_jan['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_10_jan['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][0:24]),1), mask = NA_mask_1deg_cyc )


cyc_10_feb = {}
cyc_10_feb['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][24:48]),1), mask = NA_mask_1deg_cyc  )
cyc_10_feb['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_10_feb['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_10_feb['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_10_feb['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_10_feb['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_10_feb['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_10_feb['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][24:48]),1), mask = NA_mask_1deg_cyc )


cyc_10_mar = {}
cyc_10_mar['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][48:72]),1), mask = NA_mask_1deg_cyc ) 
cyc_10_mar['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_10_mar['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_10_mar['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_10_mar['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_10_mar['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_10_mar['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_10_mar['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][48:72]),1), mask = NA_mask_1deg_cyc )


cyc_10_apr = {}
cyc_10_apr['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][72:96]),1), mask = NA_mask_1deg_cyc ) 
cyc_10_apr['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_10_apr['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_10_apr['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_10_apr['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_10_apr['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_10_apr['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_10_apr['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][72:96]),1), mask = NA_mask_1deg_cyc )


cyc_10_may = {}
cyc_10_may['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][96:120]),1), mask = NA_mask_1deg_cyc ) 
cyc_10_may['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_10_may['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_10_may['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_10_may['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_10_may['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_10_may['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_10_may['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][96:120]),1), mask = NA_mask_1deg_cyc )


cyc_10_jun = {}
cyc_10_jun['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][120:144]),1), mask = NA_mask_1deg_cyc ) 
cyc_10_jun['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_10_jun['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_10_jun['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_10_jun['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_10_jun['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_10_jun['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_10_jun['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][120:144]),1), mask = NA_mask_1deg_cyc )


cyc_10_jul = {}
cyc_10_jul['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][144:168]),1), mask = NA_mask_1deg_cyc ) 
cyc_10_jul['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_10_jul['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_10_jul['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_10_jul['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_10_jul['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_10_jul['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_10_jul['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][144:168]),1), mask = NA_mask_1deg_cyc )


cyc_10_aug = {}
cyc_10_aug['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][168:192]),1), mask = NA_mask_1deg_cyc ) 
cyc_10_aug['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_10_aug['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_10_aug['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_10_aug['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_10_aug['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_10_aug['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_10_aug['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][168:192]),1), mask = NA_mask_1deg_cyc )


cyc_10_sep = {}
cyc_10_sep['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][192:216]),1), mask = NA_mask_1deg_cyc ) 
cyc_10_sep['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_10_sep['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_10_sep['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_10_sep['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_10_sep['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_10_sep['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_10_sep['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][192:216]),1), mask = NA_mask_1deg_cyc )


cyc_10_oct = {}
cyc_10_oct['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][216:240]),1), mask = NA_mask_1deg_cyc ) 
cyc_10_oct['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_10_oct['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_10_oct['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_10_oct['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_10_oct['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_10_oct['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_10_oct['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][216:240]),1), mask = NA_mask_1deg_cyc )


cyc_10_nov = {}
cyc_10_nov['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][240:264]),1), mask = NA_mask_1deg_cyc ) 
cyc_10_nov['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_10_nov['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_10_nov['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_10_nov['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_10_nov['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_10_nov['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_10_nov['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][240:264]),1), mask = NA_mask_1deg_cyc )


cyc_10_dec = {}
cyc_10_dec['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][264:288]),1), mask = NA_mask_1deg_cyc ) 
cyc_10_dec['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_10_dec['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_10_dec['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_10_dec['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_10_dec['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_10_dec['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_10_dec['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][264:288]),1), mask = NA_mask_1deg_cyc )


h.close()


#flattening the arrays for each month and saving them as dfs
print('flattening arrays')

flat_10_jan = {}
for name,arra in cyc_10_jan.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_10_jan[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_10_jan = pd.DataFrame(flat_10_jan)
df_10_jan['year'] = 2010
df_10_jan['month'] = 'jan'

flat_10_feb = {}
for name,arra in cyc_10_feb.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_10_feb[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_10_feb = pd.DataFrame(flat_10_feb)
df_10_feb['year'] = 2010
df_10_feb['month'] = 'feb'

flat_10_mar = {}
for name,arra in cyc_10_mar.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_10_mar[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_10_mar = pd.DataFrame(flat_10_mar)
df_10_mar['year'] = 2010
df_10_mar['month'] = 'mar'

flat_10_apr = {}
for name,arra in cyc_10_apr.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_10_apr[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_10_apr = pd.DataFrame(flat_10_apr)
df_10_apr['year'] = 2010
df_10_apr['month'] = 'apr'

flat_10_may = {}
for name,arra in cyc_10_may.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_10_may[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_10_may = pd.DataFrame(flat_10_may)
df_10_may['year'] = 2010
df_10_may['month'] = 'may'

flat_10_jun = {}
for name,arra in cyc_10_jun.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_10_jun[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_10_jun = pd.DataFrame(flat_10_jun)
df_10_jun['year'] = 2010
df_10_jun['month'] = 'jun'

flat_10_jul = {}
for name,arra in cyc_10_jul.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_10_jul[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_10_jul = pd.DataFrame(flat_10_jul)
df_10_jul['year'] = 2010
df_10_jul['month'] = 'jul'

flat_10_aug = {}
for name,arra in cyc_10_aug.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_10_aug[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_10_aug = pd.DataFrame(flat_10_aug)
df_10_aug['year'] = 2010
df_10_aug['month'] = 'aug'

flat_10_sep = {}
for name,arra in cyc_10_sep.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_10_sep[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_10_sep = pd.DataFrame(flat_10_sep)
df_10_sep['year'] = 2010
df_10_sep['month'] = 'sep'

flat_10_oct = {}
for name,arra in cyc_10_oct.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_10_oct[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_10_oct = pd.DataFrame(flat_10_oct)
df_10_oct['year'] = 2010
df_10_oct['month'] = 'oct'

flat_10_nov = {}
for name,arra in cyc_10_nov.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_10_nov[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_10_nov = pd.DataFrame(flat_10_nov)
df_10_nov['year'] = 2010
df_10_nov['month'] = 'nov'

flat_10_dec = {}
for name,arra in cyc_10_dec.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_10_dec[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_10_dec = pd.DataFrame(flat_10_dec)
df_10_dec['year'] = 2010
df_10_dec['month'] = 'dec'

print('saving dataframes')
df_10_jan.to_csv('\\df_10_jan.csv', index = False)
df_10_feb.to_csv('\\df_10_feb.csv', index = False)
df_10_mar.to_csv('\\df_10_mar.csv', index = False)
df_10_apr.to_csv('\\df_10_apr.csv', index = False)
df_10_may.to_csv('\\df_10_may.csv', index = False)
df_10_jun.to_csv('\\df_10_jun.csv', index = False)
df_10_jul.to_csv('\\df_10_jul.csv', index = False)
df_10_aug.to_csv('\\df_10_aug.csv', index = False)
df_10_sep.to_csv('\\df_10_sep.csv', index = False)
df_10_oct.to_csv('\\df_10_oct.csv', index = False)
df_10_nov.to_csv('\\df_10_nov.csv', index = False)
df_10_dec.to_csv('\\df_10_dec.csv', index = False)

In [ ]:
%%time

#2009 data

#data is shape (288, 441,741)
#first 0-23 in index[0] are the diurnal cycle for January of that 

filename_ERA = "mon_cycle_"

print('Doing Dictionaries')
#doing 2009
h = nc.Dataset(filename_ERA + '2009.nc')

cyc_09_jan = {}
cyc_09_jan['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][0:24]),1), mask = NA_mask_1deg_cyc ) 
cyc_09_jan['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_09_jan['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_09_jan['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_09_jan['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_09_jan['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_09_jan['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_09_jan['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][0:24]),1), mask = NA_mask_1deg_cyc )


cyc_09_feb = {}
cyc_09_feb['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][24:48]),1), mask = NA_mask_1deg_cyc  )
cyc_09_feb['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_09_feb['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_09_feb['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_09_feb['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_09_feb['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_09_feb['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_09_feb['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][24:48]),1), mask = NA_mask_1deg_cyc )


cyc_09_mar = {}
cyc_09_mar['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][48:72]),1), mask = NA_mask_1deg_cyc ) 
cyc_09_mar['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_09_mar['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_09_mar['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_09_mar['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_09_mar['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_09_mar['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_09_mar['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][48:72]),1), mask = NA_mask_1deg_cyc )


cyc_09_apr = {}
cyc_09_apr['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][72:96]),1), mask = NA_mask_1deg_cyc ) 
cyc_09_apr['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_09_apr['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_09_apr['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_09_apr['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_09_apr['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_09_apr['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_09_apr['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][72:96]),1), mask = NA_mask_1deg_cyc )


cyc_09_may = {}
cyc_09_may['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][96:120]),1), mask = NA_mask_1deg_cyc ) 
cyc_09_may['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_09_may['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_09_may['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_09_may['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_09_may['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_09_may['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_09_may['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][96:120]),1), mask = NA_mask_1deg_cyc )


cyc_09_jun = {}
cyc_09_jun['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][120:144]),1), mask = NA_mask_1deg_cyc ) 
cyc_09_jun['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_09_jun['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_09_jun['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_09_jun['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_09_jun['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_09_jun['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_09_jun['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][120:144]),1), mask = NA_mask_1deg_cyc )


cyc_09_jul = {}
cyc_09_jul['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][144:168]),1), mask = NA_mask_1deg_cyc ) 
cyc_09_jul['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_09_jul['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_09_jul['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_09_jul['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_09_jul['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_09_jul['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_09_jul['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][144:168]),1), mask = NA_mask_1deg_cyc )


cyc_09_aug = {}
cyc_09_aug['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][168:192]),1), mask = NA_mask_1deg_cyc ) 
cyc_09_aug['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_09_aug['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_09_aug['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_09_aug['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_09_aug['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_09_aug['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_09_aug['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][168:192]),1), mask = NA_mask_1deg_cyc )


cyc_09_sep = {}
cyc_09_sep['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][192:216]),1), mask = NA_mask_1deg_cyc ) 
cyc_09_sep['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_09_sep['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_09_sep['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_09_sep['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_09_sep['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_09_sep['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_09_sep['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][192:216]),1), mask = NA_mask_1deg_cyc )


cyc_09_oct = {}
cyc_09_oct['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][216:240]),1), mask = NA_mask_1deg_cyc ) 
cyc_09_oct['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_09_oct['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_09_oct['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_09_oct['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_09_oct['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_09_oct['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_09_oct['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][216:240]),1), mask = NA_mask_1deg_cyc )


cyc_09_nov = {}
cyc_09_nov['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][240:264]),1), mask = NA_mask_1deg_cyc ) 
cyc_09_nov['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_09_nov['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_09_nov['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_09_nov['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_09_nov['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_09_nov['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_09_nov['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][240:264]),1), mask = NA_mask_1deg_cyc )


cyc_09_dec = {}
cyc_09_dec['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][264:288]),1), mask = NA_mask_1deg_cyc ) 
cyc_09_dec['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_09_dec['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_09_dec['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_09_dec['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_09_dec['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_09_dec['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_09_dec['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][264:288]),1), mask = NA_mask_1deg_cyc )


h.close()


#flattening the arrays for each month and saving them as dfs
print('flattening arrays')

flat_09_jan = {}
for name,arra in cyc_09_jan.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_09_jan[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_09_jan = pd.DataFrame(flat_09_jan)
df_09_jan['year'] = 2009
df_09_jan['month'] = 'jan'

flat_09_feb = {}
for name,arra in cyc_09_feb.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_09_feb[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_09_feb = pd.DataFrame(flat_09_feb)
df_09_feb['year'] = 2009
df_09_feb['month'] = 'feb'

flat_09_mar = {}
for name,arra in cyc_09_mar.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_09_mar[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_09_mar = pd.DataFrame(flat_09_mar)
df_09_mar['year'] = 2009
df_09_mar['month'] = 'mar'

flat_09_apr = {}
for name,arra in cyc_09_apr.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_09_apr[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_09_apr = pd.DataFrame(flat_09_apr)
df_09_apr['year'] = 2009
df_09_apr['month'] = 'apr'

flat_09_may = {}
for name,arra in cyc_09_may.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_09_may[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_09_may = pd.DataFrame(flat_09_may)
df_09_may['year'] = 2009
df_09_may['month'] = 'may'

flat_09_jun = {}
for name,arra in cyc_09_jun.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_09_jun[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_09_jun = pd.DataFrame(flat_09_jun)
df_09_jun['year'] = 2009
df_09_jun['month'] = 'jun'

flat_09_jul = {}
for name,arra in cyc_09_jul.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_09_jul[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_09_jul = pd.DataFrame(flat_09_jul)
df_09_jul['year'] = 2009
df_09_jul['month'] = 'jul'

flat_09_aug = {}
for name,arra in cyc_09_aug.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_09_aug[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_09_aug = pd.DataFrame(flat_09_aug)
df_09_aug['year'] = 2009
df_09_aug['month'] = 'aug'

flat_09_sep = {}
for name,arra in cyc_09_sep.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_09_sep[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_09_sep = pd.DataFrame(flat_09_sep)
df_09_sep['year'] = 2009
df_09_sep['month'] = 'sep'

flat_09_oct = {}
for name,arra in cyc_09_oct.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_09_oct[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_09_oct = pd.DataFrame(flat_09_oct)
df_09_oct['year'] = 2009
df_09_oct['month'] = 'oct'

flat_09_nov = {}
for name,arra in cyc_09_nov.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_09_nov[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_09_nov = pd.DataFrame(flat_09_nov)
df_09_nov['year'] = 2009
df_09_nov['month'] = 'nov'

flat_09_dec = {}
for name,arra in cyc_09_dec.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_09_dec[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_09_dec = pd.DataFrame(flat_09_dec)
df_09_dec['year'] = 2009
df_09_dec['month'] = 'dec'

print('saving dataframes')
df_09_jan.to_csv('\\df_09_jan.csv', index = False)
df_09_feb.to_csv('\\df_09_feb.csv', index = False)
df_09_mar.to_csv('\\df_09_mar.csv', index = False)
df_09_apr.to_csv('\\df_09_apr.csv', index = False)
df_09_may.to_csv('\\df_09_may.csv', index = False)
df_09_jun.to_csv('\\df_09_jun.csv', index = False)
df_09_jul.to_csv('\\df_09_jul.csv', index = False)
df_09_aug.to_csv('\\df_09_aug.csv', index = False)
df_09_sep.to_csv('\\df_09_sep.csv', index = False)
df_09_oct.to_csv('\\df_09_oct.csv', index = False)
df_09_nov.to_csv('\\df_09_nov.csv', index = False)
df_09_dec.to_csv('\\df_09_dec.csv', index = False)

In [ ]:
%%time

#2008 data

#data is shape (288, 441,741)
#first 0-23 in index[0] are the diurnal cycle for January of that 

filename_ERA = "mon_cycle_"

print('Doing Dictionaries')
#doing 2008
h = nc.Dataset(filename_ERA + '2008.nc')

cyc_08_jan = {}
cyc_08_jan['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][0:24]),1), mask = NA_mask_1deg_cyc ) 
cyc_08_jan['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_08_jan['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_08_jan['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_08_jan['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_08_jan['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_08_jan['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_08_jan['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][0:24]),1), mask = NA_mask_1deg_cyc )


cyc_08_feb = {}
cyc_08_feb['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][24:48]),1), mask = NA_mask_1deg_cyc  )
cyc_08_feb['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_08_feb['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_08_feb['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_08_feb['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_08_feb['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_08_feb['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_08_feb['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][24:48]),1), mask = NA_mask_1deg_cyc )


cyc_08_mar = {}
cyc_08_mar['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][48:72]),1), mask = NA_mask_1deg_cyc ) 
cyc_08_mar['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_08_mar['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_08_mar['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_08_mar['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_08_mar['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_08_mar['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_08_mar['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][48:72]),1), mask = NA_mask_1deg_cyc )


cyc_08_apr = {}
cyc_08_apr['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][72:96]),1), mask = NA_mask_1deg_cyc ) 
cyc_08_apr['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_08_apr['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_08_apr['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_08_apr['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_08_apr['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_08_apr['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_08_apr['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][72:96]),1), mask = NA_mask_1deg_cyc )


cyc_08_may = {}
cyc_08_may['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][96:120]),1), mask = NA_mask_1deg_cyc ) 
cyc_08_may['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_08_may['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_08_may['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_08_may['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_08_may['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_08_may['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_08_may['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][96:120]),1), mask = NA_mask_1deg_cyc )


cyc_08_jun = {}
cyc_08_jun['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][120:144]),1), mask = NA_mask_1deg_cyc ) 
cyc_08_jun['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_08_jun['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_08_jun['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_08_jun['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_08_jun['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_08_jun['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_08_jun['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][120:144]),1), mask = NA_mask_1deg_cyc )


cyc_08_jul = {}
cyc_08_jul['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][144:168]),1), mask = NA_mask_1deg_cyc ) 
cyc_08_jul['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_08_jul['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_08_jul['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_08_jul['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_08_jul['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_08_jul['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_08_jul['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][144:168]),1), mask = NA_mask_1deg_cyc )


cyc_08_aug = {}
cyc_08_aug['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][168:192]),1), mask = NA_mask_1deg_cyc ) 
cyc_08_aug['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_08_aug['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_08_aug['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_08_aug['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_08_aug['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_08_aug['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_08_aug['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][168:192]),1), mask = NA_mask_1deg_cyc )


cyc_08_sep = {}
cyc_08_sep['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][192:216]),1), mask = NA_mask_1deg_cyc ) 
cyc_08_sep['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_08_sep['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_08_sep['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_08_sep['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_08_sep['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_08_sep['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_08_sep['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][192:216]),1), mask = NA_mask_1deg_cyc )


cyc_08_oct = {}
cyc_08_oct['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][216:240]),1), mask = NA_mask_1deg_cyc ) 
cyc_08_oct['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_08_oct['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_08_oct['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_08_oct['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_08_oct['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_08_oct['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_08_oct['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][216:240]),1), mask = NA_mask_1deg_cyc )


cyc_08_nov = {}
cyc_08_nov['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][240:264]),1), mask = NA_mask_1deg_cyc ) 
cyc_08_nov['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_08_nov['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_08_nov['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_08_nov['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_08_nov['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_08_nov['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_08_nov['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][240:264]),1), mask = NA_mask_1deg_cyc )


cyc_08_dec = {}
cyc_08_dec['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][264:288]),1), mask = NA_mask_1deg_cyc ) 
cyc_08_dec['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_08_dec['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_08_dec['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_08_dec['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_08_dec['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_08_dec['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_08_dec['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][264:288]),1), mask = NA_mask_1deg_cyc )


h.close()


#flattening the arrays for each month and saving them as dfs
print('flattening arrays')

flat_08_jan = {}
for name,arra in cyc_08_jan.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_08_jan[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_08_jan = pd.DataFrame(flat_08_jan)
df_08_jan['year'] = 2008
df_08_jan['month'] = 'jan'

flat_08_feb = {}
for name,arra in cyc_08_feb.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_08_feb[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_08_feb = pd.DataFrame(flat_08_feb)
df_08_feb['year'] = 2008
df_08_feb['month'] = 'feb'

flat_08_mar = {}
for name,arra in cyc_08_mar.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_08_mar[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_08_mar = pd.DataFrame(flat_08_mar)
df_08_mar['year'] = 2008
df_08_mar['month'] = 'mar'

flat_08_apr = {}
for name,arra in cyc_08_apr.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_08_apr[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_08_apr = pd.DataFrame(flat_08_apr)
df_08_apr['year'] = 2008
df_08_apr['month'] = 'apr'

flat_08_may = {}
for name,arra in cyc_08_may.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_08_may[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_08_may = pd.DataFrame(flat_08_may)
df_08_may['year'] = 2008
df_08_may['month'] = 'may'

flat_08_jun = {}
for name,arra in cyc_08_jun.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_08_jun[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_08_jun = pd.DataFrame(flat_08_jun)
df_08_jun['year'] = 2008
df_08_jun['month'] = 'jun'

flat_08_jul = {}
for name,arra in cyc_08_jul.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_08_jul[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_08_jul = pd.DataFrame(flat_08_jul)
df_08_jul['year'] = 2008
df_08_jul['month'] = 'jul'

flat_08_aug = {}
for name,arra in cyc_08_aug.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_08_aug[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_08_aug = pd.DataFrame(flat_08_aug)
df_08_aug['year'] = 2008
df_08_aug['month'] = 'aug'

flat_08_sep = {}
for name,arra in cyc_08_sep.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_08_sep[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_08_sep = pd.DataFrame(flat_08_sep)
df_08_sep['year'] = 2008
df_08_sep['month'] = 'sep'

flat_08_oct = {}
for name,arra in cyc_08_oct.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_08_oct[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_08_oct = pd.DataFrame(flat_08_oct)
df_08_oct['year'] = 2008
df_08_oct['month'] = 'oct'

flat_08_nov = {}
for name,arra in cyc_08_nov.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_08_nov[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_08_nov = pd.DataFrame(flat_08_nov)
df_08_nov['year'] = 2008
df_08_nov['month'] = 'nov'

flat_08_dec = {}
for name,arra in cyc_08_dec.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_08_dec[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_08_dec = pd.DataFrame(flat_08_dec)
df_08_dec['year'] = 2008
df_08_dec['month'] = 'dec'

print('saving dataframes')
df_08_jan.to_csv('\\df_08_jan.csv', index = False)
df_08_feb.to_csv('\\df_08_feb.csv', index = False)
df_08_mar.to_csv('\\df_08_mar.csv', index = False)
df_08_apr.to_csv('\\df_08_apr.csv', index = False)
df_08_may.to_csv('\\df_08_may.csv', index = False)
df_08_jun.to_csv('\\df_08_jun.csv', index = False)
df_08_jul.to_csv('\\df_08_jul.csv', index = False)
df_08_aug.to_csv('\\df_08_aug.csv', index = False)
df_08_sep.to_csv('\\df_08_sep.csv', index = False)
df_08_oct.to_csv('\\df_08_oct.csv', index = False)
df_08_nov.to_csv('\\df_08_nov.csv', index = False)
df_08_dec.to_csv('\\df_08_dec.csv', index = False)

In [ ]:
%%time

#2007 data

#data is shape (288, 441,741)
#first 0-23 in index[0] are the diurnal cycle for January of that 

filename_ERA = "mon_cycle_"

print('Doing Dictionaries')
#doing 2007
h = nc.Dataset(filename_ERA + '2007.nc')

cyc_07_jan = {}
cyc_07_jan['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][0:24]),1), mask = NA_mask_1deg_cyc ) 
cyc_07_jan['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_07_jan['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_07_jan['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_07_jan['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_07_jan['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_07_jan['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_07_jan['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][0:24]),1), mask = NA_mask_1deg_cyc )


cyc_07_feb = {}
cyc_07_feb['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][24:48]),1), mask = NA_mask_1deg_cyc  )
cyc_07_feb['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_07_feb['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_07_feb['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_07_feb['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_07_feb['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_07_feb['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_07_feb['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][24:48]),1), mask = NA_mask_1deg_cyc )


cyc_07_mar = {}
cyc_07_mar['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][48:72]),1), mask = NA_mask_1deg_cyc ) 
cyc_07_mar['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_07_mar['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_07_mar['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_07_mar['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_07_mar['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_07_mar['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_07_mar['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][48:72]),1), mask = NA_mask_1deg_cyc )


cyc_07_apr = {}
cyc_07_apr['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][72:96]),1), mask = NA_mask_1deg_cyc ) 
cyc_07_apr['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_07_apr['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_07_apr['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_07_apr['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_07_apr['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_07_apr['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_07_apr['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][72:96]),1), mask = NA_mask_1deg_cyc )


cyc_07_may = {}
cyc_07_may['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][96:120]),1), mask = NA_mask_1deg_cyc ) 
cyc_07_may['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_07_may['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_07_may['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_07_may['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_07_may['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_07_may['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_07_may['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][96:120]),1), mask = NA_mask_1deg_cyc )


cyc_07_jun = {}
cyc_07_jun['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][120:144]),1), mask = NA_mask_1deg_cyc ) 
cyc_07_jun['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_07_jun['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_07_jun['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_07_jun['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_07_jun['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_07_jun['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_07_jun['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][120:144]),1), mask = NA_mask_1deg_cyc )


cyc_07_jul = {}
cyc_07_jul['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][144:168]),1), mask = NA_mask_1deg_cyc ) 
cyc_07_jul['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_07_jul['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_07_jul['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_07_jul['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_07_jul['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_07_jul['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_07_jul['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][144:168]),1), mask = NA_mask_1deg_cyc )


cyc_07_aug = {}
cyc_07_aug['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][168:192]),1), mask = NA_mask_1deg_cyc ) 
cyc_07_aug['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_07_aug['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_07_aug['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_07_aug['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_07_aug['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_07_aug['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_07_aug['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][168:192]),1), mask = NA_mask_1deg_cyc )


cyc_07_sep = {}
cyc_07_sep['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][192:216]),1), mask = NA_mask_1deg_cyc ) 
cyc_07_sep['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_07_sep['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_07_sep['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_07_sep['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_07_sep['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_07_sep['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_07_sep['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][192:216]),1), mask = NA_mask_1deg_cyc )


cyc_07_oct = {}
cyc_07_oct['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][216:240]),1), mask = NA_mask_1deg_cyc ) 
cyc_07_oct['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_07_oct['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_07_oct['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_07_oct['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_07_oct['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_07_oct['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_07_oct['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][216:240]),1), mask = NA_mask_1deg_cyc )


cyc_07_nov = {}
cyc_07_nov['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][240:264]),1), mask = NA_mask_1deg_cyc ) 
cyc_07_nov['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_07_nov['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_07_nov['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_07_nov['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_07_nov['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_07_nov['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_07_nov['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][240:264]),1), mask = NA_mask_1deg_cyc )


cyc_07_dec = {}
cyc_07_dec['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][264:288]),1), mask = NA_mask_1deg_cyc ) 
cyc_07_dec['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_07_dec['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_07_dec['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_07_dec['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_07_dec['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_07_dec['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_07_dec['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][264:288]),1), mask = NA_mask_1deg_cyc )


h.close()


#flattening the arrays for each month and saving them as dfs
print('flattening arrays')

flat_07_jan = {}
for name,arra in cyc_07_jan.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_07_jan[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_07_jan = pd.DataFrame(flat_07_jan)
df_07_jan['year'] = 2007
df_07_jan['month'] = 'jan'

flat_07_feb = {}
for name,arra in cyc_07_feb.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_07_feb[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_07_feb = pd.DataFrame(flat_07_feb)
df_07_feb['year'] = 2007
df_07_feb['month'] = 'feb'

flat_07_mar = {}
for name,arra in cyc_07_mar.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_07_mar[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_07_mar = pd.DataFrame(flat_07_mar)
df_07_mar['year'] = 2007
df_07_mar['month'] = 'mar'

flat_07_apr = {}
for name,arra in cyc_07_apr.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_07_apr[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_07_apr = pd.DataFrame(flat_07_apr)
df_07_apr['year'] = 2007
df_07_apr['month'] = 'apr'

flat_07_may = {}
for name,arra in cyc_07_may.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_07_may[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_07_may = pd.DataFrame(flat_07_may)
df_07_may['year'] = 2007
df_07_may['month'] = 'may'

flat_07_jun = {}
for name,arra in cyc_07_jun.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_07_jun[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_07_jun = pd.DataFrame(flat_07_jun)
df_07_jun['year'] = 2007
df_07_jun['month'] = 'jun'

flat_07_jul = {}
for name,arra in cyc_07_jul.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_07_jul[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_07_jul = pd.DataFrame(flat_07_jul)
df_07_jul['year'] = 2007
df_07_jul['month'] = 'jul'

flat_07_aug = {}
for name,arra in cyc_07_aug.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_07_aug[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_07_aug = pd.DataFrame(flat_07_aug)
df_07_aug['year'] = 2007
df_07_aug['month'] = 'aug'

flat_07_sep = {}
for name,arra in cyc_07_sep.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_07_sep[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_07_sep = pd.DataFrame(flat_07_sep)
df_07_sep['year'] = 2007
df_07_sep['month'] = 'sep'

flat_07_oct = {}
for name,arra in cyc_07_oct.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_07_oct[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_07_oct = pd.DataFrame(flat_07_oct)
df_07_oct['year'] = 2007
df_07_oct['month'] = 'oct'

flat_07_nov = {}
for name,arra in cyc_07_nov.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_07_nov[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_07_nov = pd.DataFrame(flat_07_nov)
df_07_nov['year'] = 2007
df_07_nov['month'] = 'nov'

flat_07_dec = {}
for name,arra in cyc_07_dec.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_07_dec[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_07_dec = pd.DataFrame(flat_07_dec)
df_07_dec['year'] = 2007
df_07_dec['month'] = 'dec'

print('saving dataframes')
df_07_jan.to_csv('\\df_07_jan.csv', index = False)
df_07_feb.to_csv('\\df_07_feb.csv', index = False)
df_07_mar.to_csv('\\df_07_mar.csv', index = False)
df_07_apr.to_csv('\\df_07_apr.csv', index = False)
df_07_may.to_csv('\\df_07_may.csv', index = False)
df_07_jun.to_csv('\\df_07_jun.csv', index = False)
df_07_jul.to_csv('\\df_07_jul.csv', index = False)
df_07_aug.to_csv('\\df_07_aug.csv', index = False)
df_07_sep.to_csv('\\df_07_sep.csv', index = False)
df_07_oct.to_csv('\\df_07_oct.csv', index = False)
df_07_nov.to_csv('\\df_07_nov.csv', index = False)
df_07_dec.to_csv('\\df_07_dec.csv', index = False)

In [ ]:
%%time

#2006 data

#data is shape (288, 441,741)
#first 0-23 in index[0] are the diurnal cycle for January of that 

filename_ERA = "mon_cycle_"

print('Doing Dictionaries')
#doing 2006
h = nc.Dataset(filename_ERA + '2006.nc')

cyc_06_jan = {}
cyc_06_jan['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][0:24]),1), mask = NA_mask_1deg_cyc ) 
cyc_06_jan['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_06_jan['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_06_jan['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_06_jan['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_06_jan['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_06_jan['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_06_jan['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][0:24]),1), mask = NA_mask_1deg_cyc )


cyc_06_feb = {}
cyc_06_feb['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][24:48]),1), mask = NA_mask_1deg_cyc  )
cyc_06_feb['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_06_feb['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_06_feb['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_06_feb['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_06_feb['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_06_feb['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_06_feb['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][24:48]),1), mask = NA_mask_1deg_cyc )


cyc_06_mar = {}
cyc_06_mar['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][48:72]),1), mask = NA_mask_1deg_cyc ) 
cyc_06_mar['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_06_mar['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_06_mar['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_06_mar['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_06_mar['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_06_mar['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_06_mar['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][48:72]),1), mask = NA_mask_1deg_cyc )


cyc_06_apr = {}
cyc_06_apr['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][72:96]),1), mask = NA_mask_1deg_cyc ) 
cyc_06_apr['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_06_apr['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_06_apr['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_06_apr['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_06_apr['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_06_apr['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_06_apr['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][72:96]),1), mask = NA_mask_1deg_cyc )


cyc_06_may = {}
cyc_06_may['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][96:120]),1), mask = NA_mask_1deg_cyc ) 
cyc_06_may['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_06_may['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_06_may['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_06_may['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_06_may['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_06_may['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_06_may['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][96:120]),1), mask = NA_mask_1deg_cyc )


cyc_06_jun = {}
cyc_06_jun['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][120:144]),1), mask = NA_mask_1deg_cyc ) 
cyc_06_jun['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_06_jun['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_06_jun['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_06_jun['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_06_jun['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_06_jun['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_06_jun['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][120:144]),1), mask = NA_mask_1deg_cyc )


cyc_06_jul = {}
cyc_06_jul['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][144:168]),1), mask = NA_mask_1deg_cyc ) 
cyc_06_jul['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_06_jul['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_06_jul['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_06_jul['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_06_jul['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_06_jul['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_06_jul['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][144:168]),1), mask = NA_mask_1deg_cyc )


cyc_06_aug = {}
cyc_06_aug['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][168:192]),1), mask = NA_mask_1deg_cyc ) 
cyc_06_aug['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_06_aug['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_06_aug['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_06_aug['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_06_aug['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_06_aug['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_06_aug['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][168:192]),1), mask = NA_mask_1deg_cyc )


cyc_06_sep = {}
cyc_06_sep['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][192:216]),1), mask = NA_mask_1deg_cyc ) 
cyc_06_sep['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_06_sep['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_06_sep['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_06_sep['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_06_sep['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_06_sep['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_06_sep['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][192:216]),1), mask = NA_mask_1deg_cyc )


cyc_06_oct = {}
cyc_06_oct['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][216:240]),1), mask = NA_mask_1deg_cyc ) 
cyc_06_oct['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_06_oct['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_06_oct['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_06_oct['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_06_oct['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_06_oct['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_06_oct['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][216:240]),1), mask = NA_mask_1deg_cyc )


cyc_06_nov = {}
cyc_06_nov['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][240:264]),1), mask = NA_mask_1deg_cyc ) 
cyc_06_nov['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_06_nov['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_06_nov['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_06_nov['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_06_nov['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_06_nov['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_06_nov['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][240:264]),1), mask = NA_mask_1deg_cyc )


cyc_06_dec = {}
cyc_06_dec['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][264:288]),1), mask = NA_mask_1deg_cyc ) 
cyc_06_dec['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_06_dec['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_06_dec['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_06_dec['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_06_dec['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_06_dec['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_06_dec['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][264:288]),1), mask = NA_mask_1deg_cyc )


h.close()


#flattening the arrays for each month and saving them as dfs
print('flattening arrays')

flat_06_jan = {}
for name,arra in cyc_06_jan.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_06_jan[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_06_jan = pd.DataFrame(flat_06_jan)
df_06_jan['year'] = 2006
df_06_jan['month'] = 'jan'

flat_06_feb = {}
for name,arra in cyc_06_feb.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_06_feb[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_06_feb = pd.DataFrame(flat_06_feb)
df_06_feb['year'] = 2006
df_06_feb['month'] = 'feb'

flat_06_mar = {}
for name,arra in cyc_06_mar.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_06_mar[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_06_mar = pd.DataFrame(flat_06_mar)
df_06_mar['year'] = 2006
df_06_mar['month'] = 'mar'

flat_06_apr = {}
for name,arra in cyc_06_apr.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_06_apr[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_06_apr = pd.DataFrame(flat_06_apr)
df_06_apr['year'] = 2006
df_06_apr['month'] = 'apr'

flat_06_may = {}
for name,arra in cyc_06_may.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_06_may[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_06_may = pd.DataFrame(flat_06_may)
df_06_may['year'] = 2006
df_06_may['month'] = 'may'

flat_06_jun = {}
for name,arra in cyc_06_jun.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_06_jun[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_06_jun = pd.DataFrame(flat_06_jun)
df_06_jun['year'] = 2006
df_06_jun['month'] = 'jun'

flat_06_jul = {}
for name,arra in cyc_06_jul.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_06_jul[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_06_jul = pd.DataFrame(flat_06_jul)
df_06_jul['year'] = 2006
df_06_jul['month'] = 'jul'

flat_06_aug = {}
for name,arra in cyc_06_aug.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_06_aug[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_06_aug = pd.DataFrame(flat_06_aug)
df_06_aug['year'] = 2006
df_06_aug['month'] = 'aug'

flat_06_sep = {}
for name,arra in cyc_06_sep.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_06_sep[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_06_sep = pd.DataFrame(flat_06_sep)
df_06_sep['year'] = 2006
df_06_sep['month'] = 'sep'

flat_06_oct = {}
for name,arra in cyc_06_oct.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_06_oct[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_06_oct = pd.DataFrame(flat_06_oct)
df_06_oct['year'] = 2006
df_06_oct['month'] = 'oct'

flat_06_nov = {}
for name,arra in cyc_06_nov.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_06_nov[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_06_nov = pd.DataFrame(flat_06_nov)
df_06_nov['year'] = 2006
df_06_nov['month'] = 'nov'

flat_06_dec = {}
for name,arra in cyc_06_dec.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_06_dec[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_06_dec = pd.DataFrame(flat_06_dec)
df_06_dec['year'] = 2006
df_06_dec['month'] = 'dec'

print('saving dataframes')
df_06_jan.to_csv('\\df_06_jan.csv', index = False)
df_06_feb.to_csv('\\df_06_feb.csv', index = False)
df_06_mar.to_csv('\\df_06_mar.csv', index = False)
df_06_apr.to_csv('\\df_06_apr.csv', index = False)
df_06_may.to_csv('\\df_06_may.csv', index = False)
df_06_jun.to_csv('\\df_06_jun.csv', index = False)
df_06_jul.to_csv('\\df_06_jul.csv', index = False)
df_06_aug.to_csv('\\df_06_aug.csv', index = False)
df_06_sep.to_csv('\\df_06_sep.csv', index = False)
df_06_oct.to_csv('\\df_06_oct.csv', index = False)
df_06_nov.to_csv('\\df_06_nov.csv', index = False)
df_06_dec.to_csv('\\df_06_dec.csv', index = False)

In [ ]:
%%time

#2005 data

#data is shape (288, 441,741)
#first 0-23 in index[0] are the diurnal cycle for January of that 

filename_ERA = "mon_cycle_"

print('Doing Dictionaries')
#doing 2005
h = nc.Dataset(filename_ERA + '2005.nc')

cyc_05_jan = {}
cyc_05_jan['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][0:24]),1), mask = NA_mask_1deg_cyc ) 
cyc_05_jan['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_05_jan['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_05_jan['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_05_jan['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_05_jan['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_05_jan['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_05_jan['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][0:24]),1), mask = NA_mask_1deg_cyc )


cyc_05_feb = {}
cyc_05_feb['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][24:48]),1), mask = NA_mask_1deg_cyc  )
cyc_05_feb['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_05_feb['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_05_feb['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_05_feb['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_05_feb['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_05_feb['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_05_feb['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][24:48]),1), mask = NA_mask_1deg_cyc )


cyc_05_mar = {}
cyc_05_mar['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][48:72]),1), mask = NA_mask_1deg_cyc ) 
cyc_05_mar['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_05_mar['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_05_mar['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_05_mar['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_05_mar['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_05_mar['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_05_mar['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][48:72]),1), mask = NA_mask_1deg_cyc )


cyc_05_apr = {}
cyc_05_apr['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][72:96]),1), mask = NA_mask_1deg_cyc ) 
cyc_05_apr['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_05_apr['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_05_apr['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_05_apr['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_05_apr['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_05_apr['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_05_apr['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][72:96]),1), mask = NA_mask_1deg_cyc )


cyc_05_may = {}
cyc_05_may['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][96:120]),1), mask = NA_mask_1deg_cyc ) 
cyc_05_may['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_05_may['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_05_may['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_05_may['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_05_may['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_05_may['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_05_may['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][96:120]),1), mask = NA_mask_1deg_cyc )


cyc_05_jun = {}
cyc_05_jun['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][120:144]),1), mask = NA_mask_1deg_cyc ) 
cyc_05_jun['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_05_jun['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_05_jun['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_05_jun['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_05_jun['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_05_jun['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_05_jun['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][120:144]),1), mask = NA_mask_1deg_cyc )


cyc_05_jul = {}
cyc_05_jul['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][144:168]),1), mask = NA_mask_1deg_cyc ) 
cyc_05_jul['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_05_jul['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_05_jul['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_05_jul['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_05_jul['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_05_jul['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_05_jul['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][144:168]),1), mask = NA_mask_1deg_cyc )


cyc_05_aug = {}
cyc_05_aug['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][168:192]),1), mask = NA_mask_1deg_cyc ) 
cyc_05_aug['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_05_aug['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_05_aug['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_05_aug['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_05_aug['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_05_aug['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_05_aug['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][168:192]),1), mask = NA_mask_1deg_cyc )


cyc_05_sep = {}
cyc_05_sep['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][192:216]),1), mask = NA_mask_1deg_cyc ) 
cyc_05_sep['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_05_sep['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_05_sep['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_05_sep['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_05_sep['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_05_sep['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_05_sep['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][192:216]),1), mask = NA_mask_1deg_cyc )


cyc_05_oct = {}
cyc_05_oct['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][216:240]),1), mask = NA_mask_1deg_cyc ) 
cyc_05_oct['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_05_oct['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_05_oct['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_05_oct['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_05_oct['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_05_oct['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_05_oct['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][216:240]),1), mask = NA_mask_1deg_cyc )


cyc_05_nov = {}
cyc_05_nov['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][240:264]),1), mask = NA_mask_1deg_cyc ) 
cyc_05_nov['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_05_nov['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_05_nov['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_05_nov['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_05_nov['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_05_nov['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_05_nov['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][240:264]),1), mask = NA_mask_1deg_cyc )


cyc_05_dec = {}
cyc_05_dec['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][264:288]),1), mask = NA_mask_1deg_cyc ) 
cyc_05_dec['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_05_dec['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_05_dec['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_05_dec['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_05_dec['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_05_dec['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_05_dec['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][264:288]),1), mask = NA_mask_1deg_cyc )


h.close()


#flattening the arrays for each month and saving them as dfs
print('flattening arrays')

flat_05_jan = {}
for name,arra in cyc_05_jan.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_05_jan[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_05_jan = pd.DataFrame(flat_05_jan)
df_05_jan['year'] = 2005
df_05_jan['month'] = 'jan'

flat_05_feb = {}
for name,arra in cyc_05_feb.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_05_feb[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_05_feb = pd.DataFrame(flat_05_feb)
df_05_feb['year'] = 2005
df_05_feb['month'] = 'feb'

flat_05_mar = {}
for name,arra in cyc_05_mar.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_05_mar[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_05_mar = pd.DataFrame(flat_05_mar)
df_05_mar['year'] = 2005
df_05_mar['month'] = 'mar'

flat_05_apr = {}
for name,arra in cyc_05_apr.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_05_apr[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_05_apr = pd.DataFrame(flat_05_apr)
df_05_apr['year'] = 2005
df_05_apr['month'] = 'apr'

flat_05_may = {}
for name,arra in cyc_05_may.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_05_may[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_05_may = pd.DataFrame(flat_05_may)
df_05_may['year'] = 2005
df_05_may['month'] = 'may'

flat_05_jun = {}
for name,arra in cyc_05_jun.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_05_jun[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_05_jun = pd.DataFrame(flat_05_jun)
df_05_jun['year'] = 2005
df_05_jun['month'] = 'jun'

flat_05_jul = {}
for name,arra in cyc_05_jul.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_05_jul[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_05_jul = pd.DataFrame(flat_05_jul)
df_05_jul['year'] = 2005
df_05_jul['month'] = 'jul'

flat_05_aug = {}
for name,arra in cyc_05_aug.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_05_aug[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_05_aug = pd.DataFrame(flat_05_aug)
df_05_aug['year'] = 2005
df_05_aug['month'] = 'aug'

flat_05_sep = {}
for name,arra in cyc_05_sep.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_05_sep[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_05_sep = pd.DataFrame(flat_05_sep)
df_05_sep['year'] = 2005
df_05_sep['month'] = 'sep'

flat_05_oct = {}
for name,arra in cyc_05_oct.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_05_oct[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_05_oct = pd.DataFrame(flat_05_oct)
df_05_oct['year'] = 2005
df_05_oct['month'] = 'oct'

flat_05_nov = {}
for name,arra in cyc_05_nov.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_05_nov[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_05_nov = pd.DataFrame(flat_05_nov)
df_05_nov['year'] = 2005
df_05_nov['month'] = 'nov'

flat_05_dec = {}
for name,arra in cyc_05_dec.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_05_dec[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_05_dec = pd.DataFrame(flat_05_dec)
df_05_dec['year'] = 2005
df_05_dec['month'] = 'dec'

print('saving dataframes')
df_05_jan.to_csv('\\df_05_jan.csv', index = False)
df_05_feb.to_csv('\\df_05_feb.csv', index = False)
df_05_mar.to_csv('\\df_05_mar.csv', index = False)
df_05_apr.to_csv('\\df_05_apr.csv', index = False)
df_05_may.to_csv('\\df_05_may.csv', index = False)
df_05_jun.to_csv('\\df_05_jun.csv', index = False)
df_05_jul.to_csv('\\df_05_jul.csv', index = False)
df_05_aug.to_csv('\\df_05_aug.csv', index = False)
df_05_sep.to_csv('\\df_05_sep.csv', index = False)
df_05_oct.to_csv('\\df_05_oct.csv', index = False)
df_05_nov.to_csv('\\df_05_nov.csv', index = False)
df_05_dec.to_csv('\\df_05_dec.csv', index = False)

In [ ]:
%%time

#2004 data

#data is shape (288, 441,741)
#first 0-23 in index[0] are the diurnal cycle for January of that 

filename_ERA = "mon_cycle_"

print('Doing Dictionaries')
#doing 2004
h = nc.Dataset(filename_ERA + '2004.nc')

cyc_04_jan = {}
cyc_04_jan['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][0:24]),1), mask = NA_mask_1deg_cyc ) 
cyc_04_jan['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_04_jan['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_04_jan['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_04_jan['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_04_jan['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_04_jan['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_04_jan['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][0:24]),1), mask = NA_mask_1deg_cyc )


cyc_04_feb = {}
cyc_04_feb['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][24:48]),1), mask = NA_mask_1deg_cyc  )
cyc_04_feb['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_04_feb['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_04_feb['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_04_feb['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_04_feb['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_04_feb['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_04_feb['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][24:48]),1), mask = NA_mask_1deg_cyc )


cyc_04_mar = {}
cyc_04_mar['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][48:72]),1), mask = NA_mask_1deg_cyc ) 
cyc_04_mar['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_04_mar['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_04_mar['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_04_mar['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_04_mar['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_04_mar['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_04_mar['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][48:72]),1), mask = NA_mask_1deg_cyc )


cyc_04_apr = {}
cyc_04_apr['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][72:96]),1), mask = NA_mask_1deg_cyc ) 
cyc_04_apr['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_04_apr['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_04_apr['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_04_apr['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_04_apr['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_04_apr['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_04_apr['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][72:96]),1), mask = NA_mask_1deg_cyc )


cyc_04_may = {}
cyc_04_may['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][96:120]),1), mask = NA_mask_1deg_cyc ) 
cyc_04_may['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_04_may['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_04_may['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_04_may['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_04_may['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_04_may['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_04_may['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][96:120]),1), mask = NA_mask_1deg_cyc )


cyc_04_jun = {}
cyc_04_jun['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][120:144]),1), mask = NA_mask_1deg_cyc ) 
cyc_04_jun['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_04_jun['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_04_jun['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_04_jun['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_04_jun['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_04_jun['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_04_jun['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][120:144]),1), mask = NA_mask_1deg_cyc )


cyc_04_jul = {}
cyc_04_jul['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][144:168]),1), mask = NA_mask_1deg_cyc ) 
cyc_04_jul['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_04_jul['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_04_jul['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_04_jul['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_04_jul['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_04_jul['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_04_jul['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][144:168]),1), mask = NA_mask_1deg_cyc )


cyc_04_aug = {}
cyc_04_aug['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][168:192]),1), mask = NA_mask_1deg_cyc ) 
cyc_04_aug['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_04_aug['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_04_aug['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_04_aug['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_04_aug['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_04_aug['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_04_aug['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][168:192]),1), mask = NA_mask_1deg_cyc )


cyc_04_sep = {}
cyc_04_sep['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][192:216]),1), mask = NA_mask_1deg_cyc ) 
cyc_04_sep['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_04_sep['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_04_sep['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_04_sep['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_04_sep['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_04_sep['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_04_sep['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][192:216]),1), mask = NA_mask_1deg_cyc )


cyc_04_oct = {}
cyc_04_oct['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][216:240]),1), mask = NA_mask_1deg_cyc ) 
cyc_04_oct['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_04_oct['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_04_oct['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_04_oct['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_04_oct['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_04_oct['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_04_oct['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][216:240]),1), mask = NA_mask_1deg_cyc )


cyc_04_nov = {}
cyc_04_nov['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][240:264]),1), mask = NA_mask_1deg_cyc ) 
cyc_04_nov['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_04_nov['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_04_nov['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_04_nov['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_04_nov['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_04_nov['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_04_nov['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][240:264]),1), mask = NA_mask_1deg_cyc )


cyc_04_dec = {}
cyc_04_dec['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][264:288]),1), mask = NA_mask_1deg_cyc ) 
cyc_04_dec['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_04_dec['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_04_dec['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_04_dec['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_04_dec['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_04_dec['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_04_dec['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][264:288]),1), mask = NA_mask_1deg_cyc )


h.close()


#flattening the arrays for each month and saving them as dfs
print('flattening arrays')

flat_04_jan = {}
for name,arra in cyc_04_jan.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_04_jan[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_04_jan = pd.DataFrame(flat_04_jan)
df_04_jan['year'] = 2004
df_04_jan['month'] = 'jan'

flat_04_feb = {}
for name,arra in cyc_04_feb.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_04_feb[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_04_feb = pd.DataFrame(flat_04_feb)
df_04_feb['year'] = 2004
df_04_feb['month'] = 'feb'

flat_04_mar = {}
for name,arra in cyc_04_mar.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_04_mar[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_04_mar = pd.DataFrame(flat_04_mar)
df_04_mar['year'] = 2004
df_04_mar['month'] = 'mar'

flat_04_apr = {}
for name,arra in cyc_04_apr.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_04_apr[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_04_apr = pd.DataFrame(flat_04_apr)
df_04_apr['year'] = 2004
df_04_apr['month'] = 'apr'

flat_04_may = {}
for name,arra in cyc_04_may.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_04_may[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_04_may = pd.DataFrame(flat_04_may)
df_04_may['year'] = 2004
df_04_may['month'] = 'may'

flat_04_jun = {}
for name,arra in cyc_04_jun.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_04_jun[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_04_jun = pd.DataFrame(flat_04_jun)
df_04_jun['year'] = 2004
df_04_jun['month'] = 'jun'

flat_04_jul = {}
for name,arra in cyc_04_jul.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_04_jul[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_04_jul = pd.DataFrame(flat_04_jul)
df_04_jul['year'] = 2004
df_04_jul['month'] = 'jul'

flat_04_aug = {}
for name,arra in cyc_04_aug.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_04_aug[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_04_aug = pd.DataFrame(flat_04_aug)
df_04_aug['year'] = 2004
df_04_aug['month'] = 'aug'

flat_04_sep = {}
for name,arra in cyc_04_sep.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_04_sep[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_04_sep = pd.DataFrame(flat_04_sep)
df_04_sep['year'] = 2004
df_04_sep['month'] = 'sep'

flat_04_oct = {}
for name,arra in cyc_04_oct.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_04_oct[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_04_oct = pd.DataFrame(flat_04_oct)
df_04_oct['year'] = 2004
df_04_oct['month'] = 'oct'

flat_04_nov = {}
for name,arra in cyc_04_nov.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_04_nov[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_04_nov = pd.DataFrame(flat_04_nov)
df_04_nov['year'] = 2004
df_04_nov['month'] = 'nov'

flat_04_dec = {}
for name,arra in cyc_04_dec.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_04_dec[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_04_dec = pd.DataFrame(flat_04_dec)
df_04_dec['year'] = 2004
df_04_dec['month'] = 'dec'

print('saving dataframes')
df_04_jan.to_csv('\\df_04_jan.csv', index = False)
df_04_feb.to_csv('\\df_04_feb.csv', index = False)
df_04_mar.to_csv('\\df_04_mar.csv', index = False)
df_04_apr.to_csv('\\df_04_apr.csv', index = False)
df_04_may.to_csv('\\df_04_may.csv', index = False)
df_04_jun.to_csv('\\df_04_jun.csv', index = False)
df_04_jul.to_csv('\\df_04_jul.csv', index = False)
df_04_aug.to_csv('\\df_04_aug.csv', index = False)
df_04_sep.to_csv('\\df_04_sep.csv', index = False)
df_04_oct.to_csv('\\df_04_oct.csv', index = False)
df_04_nov.to_csv('\\df_04_nov.csv', index = False)
df_04_dec.to_csv('\\df_04_dec.csv', index = False)

In [ ]:
%%time

#2003 data

#data is shape (288, 441,741)
#first 0-23 in index[0] are the diurnal cycle for January of that 

filename_ERA = "mon_cycle_"

print('Doing Dictionaries')
#doing 2003
h = nc.Dataset(filename_ERA + '2003.nc')

cyc_03_jan = {}
cyc_03_jan['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][0:24]),1), mask = NA_mask_1deg_cyc ) 
cyc_03_jan['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_03_jan['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_03_jan['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_03_jan['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_03_jan['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_03_jan['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_03_jan['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][0:24]),1), mask = NA_mask_1deg_cyc )


cyc_03_feb = {}
cyc_03_feb['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][24:48]),1), mask = NA_mask_1deg_cyc  )
cyc_03_feb['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_03_feb['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_03_feb['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_03_feb['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_03_feb['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_03_feb['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_03_feb['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][24:48]),1), mask = NA_mask_1deg_cyc )


cyc_03_mar = {}
cyc_03_mar['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][48:72]),1), mask = NA_mask_1deg_cyc ) 
cyc_03_mar['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_03_mar['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_03_mar['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_03_mar['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_03_mar['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_03_mar['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_03_mar['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][48:72]),1), mask = NA_mask_1deg_cyc )


cyc_03_apr = {}
cyc_03_apr['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][72:96]),1), mask = NA_mask_1deg_cyc ) 
cyc_03_apr['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_03_apr['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_03_apr['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_03_apr['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_03_apr['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_03_apr['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_03_apr['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][72:96]),1), mask = NA_mask_1deg_cyc )


cyc_03_may = {}
cyc_03_may['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][96:120]),1), mask = NA_mask_1deg_cyc ) 
cyc_03_may['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_03_may['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_03_may['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_03_may['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_03_may['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_03_may['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_03_may['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][96:120]),1), mask = NA_mask_1deg_cyc )


cyc_03_jun = {}
cyc_03_jun['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][120:144]),1), mask = NA_mask_1deg_cyc ) 
cyc_03_jun['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_03_jun['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_03_jun['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_03_jun['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_03_jun['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_03_jun['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_03_jun['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][120:144]),1), mask = NA_mask_1deg_cyc )


cyc_03_jul = {}
cyc_03_jul['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][144:168]),1), mask = NA_mask_1deg_cyc ) 
cyc_03_jul['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_03_jul['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_03_jul['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_03_jul['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_03_jul['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_03_jul['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_03_jul['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][144:168]),1), mask = NA_mask_1deg_cyc )


cyc_03_aug = {}
cyc_03_aug['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][168:192]),1), mask = NA_mask_1deg_cyc ) 
cyc_03_aug['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_03_aug['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_03_aug['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_03_aug['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_03_aug['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_03_aug['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_03_aug['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][168:192]),1), mask = NA_mask_1deg_cyc )


cyc_03_sep = {}
cyc_03_sep['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][192:216]),1), mask = NA_mask_1deg_cyc ) 
cyc_03_sep['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_03_sep['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_03_sep['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_03_sep['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_03_sep['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_03_sep['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_03_sep['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][192:216]),1), mask = NA_mask_1deg_cyc )


cyc_03_oct = {}
cyc_03_oct['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][216:240]),1), mask = NA_mask_1deg_cyc ) 
cyc_03_oct['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_03_oct['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_03_oct['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_03_oct['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_03_oct['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_03_oct['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_03_oct['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][216:240]),1), mask = NA_mask_1deg_cyc )


cyc_03_nov = {}
cyc_03_nov['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][240:264]),1), mask = NA_mask_1deg_cyc ) 
cyc_03_nov['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_03_nov['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_03_nov['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_03_nov['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_03_nov['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_03_nov['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_03_nov['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][240:264]),1), mask = NA_mask_1deg_cyc )


cyc_03_dec = {}
cyc_03_dec['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][264:288]),1), mask = NA_mask_1deg_cyc ) 
cyc_03_dec['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_03_dec['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_03_dec['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_03_dec['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_03_dec['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_03_dec['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_03_dec['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][264:288]),1), mask = NA_mask_1deg_cyc )


h.close()


#flattening the arrays for each month and saving them as dfs
print('flattening arrays')

flat_03_jan = {}
for name,arra in cyc_03_jan.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_03_jan[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_03_jan = pd.DataFrame(flat_03_jan)
df_03_jan['year'] = 2003
df_03_jan['month'] = 'jan'

flat_03_feb = {}
for name,arra in cyc_03_feb.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_03_feb[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_03_feb = pd.DataFrame(flat_03_feb)
df_03_feb['year'] = 2003
df_03_feb['month'] = 'feb'

flat_03_mar = {}
for name,arra in cyc_03_mar.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_03_mar[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_03_mar = pd.DataFrame(flat_03_mar)
df_03_mar['year'] = 2003
df_03_mar['month'] = 'mar'

flat_03_apr = {}
for name,arra in cyc_03_apr.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_03_apr[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_03_apr = pd.DataFrame(flat_03_apr)
df_03_apr['year'] = 2003
df_03_apr['month'] = 'apr'

flat_03_may = {}
for name,arra in cyc_03_may.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_03_may[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_03_may = pd.DataFrame(flat_03_may)
df_03_may['year'] = 2003
df_03_may['month'] = 'may'

flat_03_jun = {}
for name,arra in cyc_03_jun.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_03_jun[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_03_jun = pd.DataFrame(flat_03_jun)
df_03_jun['year'] = 2003
df_03_jun['month'] = 'jun'

flat_03_jul = {}
for name,arra in cyc_03_jul.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_03_jul[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_03_jul = pd.DataFrame(flat_03_jul)
df_03_jul['year'] = 2003
df_03_jul['month'] = 'jul'

flat_03_aug = {}
for name,arra in cyc_03_aug.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_03_aug[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_03_aug = pd.DataFrame(flat_03_aug)
df_03_aug['year'] = 2003
df_03_aug['month'] = 'aug'

flat_03_sep = {}
for name,arra in cyc_03_sep.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_03_sep[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_03_sep = pd.DataFrame(flat_03_sep)
df_03_sep['year'] = 2003
df_03_sep['month'] = 'sep'

flat_03_oct = {}
for name,arra in cyc_03_oct.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_03_oct[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_03_oct = pd.DataFrame(flat_03_oct)
df_03_oct['year'] = 2003
df_03_oct['month'] = 'oct'

flat_03_nov = {}
for name,arra in cyc_03_nov.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_03_nov[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_03_nov = pd.DataFrame(flat_03_nov)
df_03_nov['year'] = 2003
df_03_nov['month'] = 'nov'

flat_03_dec = {}
for name,arra in cyc_03_dec.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_03_dec[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_03_dec = pd.DataFrame(flat_03_dec)
df_03_dec['year'] = 2003
df_03_dec['month'] = 'dec'

print('saving dataframes')
df_03_jan.to_csv('\\df_03_jan.csv', index = False)
df_03_feb.to_csv('\\df_03_feb.csv', index = False)
df_03_mar.to_csv('\\df_03_mar.csv', index = False)
df_03_apr.to_csv('\\df_03_apr.csv', index = False)
df_03_may.to_csv('\\df_03_may.csv', index = False)
df_03_jun.to_csv('\\df_03_jun.csv', index = False)
df_03_jul.to_csv('\\df_03_jul.csv', index = False)
df_03_aug.to_csv('\\df_03_aug.csv', index = False)
df_03_sep.to_csv('\\df_03_sep.csv', index = False)
df_03_oct.to_csv('\\df_03_oct.csv', index = False)
df_03_nov.to_csv('\\df_03_nov.csv', index = False)
df_03_dec.to_csv('\\df_03_dec.csv', index = False)

In [ ]:
%%time

#2002 data

#data is shape (288, 441,741)
#first 0-23 in index[0] are the diurnal cycle for January of that 

filename_ERA = "mon_cycle_"

print('Doing Dictionaries')
#doing 2002
h = nc.Dataset(filename_ERA + '2002.nc')

cyc_02_jan = {}
cyc_02_jan['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][0:24]),1), mask = NA_mask_1deg_cyc ) 
cyc_02_jan['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_02_jan['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_02_jan['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_02_jan['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_02_jan['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_02_jan['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_02_jan['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][0:24]),1), mask = NA_mask_1deg_cyc )


cyc_02_feb = {}
cyc_02_feb['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][24:48]),1), mask = NA_mask_1deg_cyc  )
cyc_02_feb['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_02_feb['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_02_feb['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_02_feb['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_02_feb['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_02_feb['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_02_feb['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][24:48]),1), mask = NA_mask_1deg_cyc )


cyc_02_mar = {}
cyc_02_mar['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][48:72]),1), mask = NA_mask_1deg_cyc ) 
cyc_02_mar['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_02_mar['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_02_mar['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_02_mar['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_02_mar['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_02_mar['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_02_mar['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][48:72]),1), mask = NA_mask_1deg_cyc )


cyc_02_apr = {}
cyc_02_apr['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][72:96]),1), mask = NA_mask_1deg_cyc ) 
cyc_02_apr['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_02_apr['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_02_apr['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_02_apr['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_02_apr['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_02_apr['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_02_apr['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][72:96]),1), mask = NA_mask_1deg_cyc )


cyc_02_may = {}
cyc_02_may['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][96:120]),1), mask = NA_mask_1deg_cyc ) 
cyc_02_may['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_02_may['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_02_may['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_02_may['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_02_may['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_02_may['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_02_may['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][96:120]),1), mask = NA_mask_1deg_cyc )


cyc_02_jun = {}
cyc_02_jun['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][120:144]),1), mask = NA_mask_1deg_cyc ) 
cyc_02_jun['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_02_jun['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_02_jun['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_02_jun['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_02_jun['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_02_jun['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_02_jun['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][120:144]),1), mask = NA_mask_1deg_cyc )


cyc_02_jul = {}
cyc_02_jul['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][144:168]),1), mask = NA_mask_1deg_cyc ) 
cyc_02_jul['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_02_jul['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_02_jul['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_02_jul['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_02_jul['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_02_jul['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_02_jul['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][144:168]),1), mask = NA_mask_1deg_cyc )


cyc_02_aug = {}
cyc_02_aug['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][168:192]),1), mask = NA_mask_1deg_cyc ) 
cyc_02_aug['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_02_aug['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_02_aug['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_02_aug['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_02_aug['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_02_aug['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_02_aug['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][168:192]),1), mask = NA_mask_1deg_cyc )


cyc_02_sep = {}
cyc_02_sep['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][192:216]),1), mask = NA_mask_1deg_cyc ) 
cyc_02_sep['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_02_sep['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_02_sep['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_02_sep['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_02_sep['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_02_sep['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_02_sep['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][192:216]),1), mask = NA_mask_1deg_cyc )


cyc_02_oct = {}
cyc_02_oct['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][216:240]),1), mask = NA_mask_1deg_cyc ) 
cyc_02_oct['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_02_oct['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_02_oct['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_02_oct['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_02_oct['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_02_oct['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_02_oct['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][216:240]),1), mask = NA_mask_1deg_cyc )


cyc_02_nov = {}
cyc_02_nov['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][240:264]),1), mask = NA_mask_1deg_cyc ) 
cyc_02_nov['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_02_nov['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_02_nov['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_02_nov['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_02_nov['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_02_nov['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_02_nov['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][240:264]),1), mask = NA_mask_1deg_cyc )


cyc_02_dec = {}
cyc_02_dec['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][264:288]),1), mask = NA_mask_1deg_cyc ) 
cyc_02_dec['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_02_dec['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_02_dec['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_02_dec['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_02_dec['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_02_dec['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_02_dec['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][264:288]),1), mask = NA_mask_1deg_cyc )


h.close()


#flattening the arrays for each month and saving them as dfs
print('flattening arrays')

flat_02_jan = {}
for name,arra in cyc_02_jan.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_02_jan[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_02_jan = pd.DataFrame(flat_02_jan)
df_02_jan['year'] = 2002
df_02_jan['month'] = 'jan'

flat_02_feb = {}
for name,arra in cyc_02_feb.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_02_feb[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_02_feb = pd.DataFrame(flat_02_feb)
df_02_feb['year'] = 2002
df_02_feb['month'] = 'feb'

flat_02_mar = {}
for name,arra in cyc_02_mar.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_02_mar[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_02_mar = pd.DataFrame(flat_02_mar)
df_02_mar['year'] = 2002
df_02_mar['month'] = 'mar'

flat_02_apr = {}
for name,arra in cyc_02_apr.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_02_apr[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_02_apr = pd.DataFrame(flat_02_apr)
df_02_apr['year'] = 2002
df_02_apr['month'] = 'apr'

flat_02_may = {}
for name,arra in cyc_02_may.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_02_may[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_02_may = pd.DataFrame(flat_02_may)
df_02_may['year'] = 2002
df_02_may['month'] = 'may'

flat_02_jun = {}
for name,arra in cyc_02_jun.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_02_jun[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_02_jun = pd.DataFrame(flat_02_jun)
df_02_jun['year'] = 2002
df_02_jun['month'] = 'jun'

flat_02_jul = {}
for name,arra in cyc_02_jul.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_02_jul[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_02_jul = pd.DataFrame(flat_02_jul)
df_02_jul['year'] = 2002
df_02_jul['month'] = 'jul'

flat_02_aug = {}
for name,arra in cyc_02_aug.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_02_aug[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_02_aug = pd.DataFrame(flat_02_aug)
df_02_aug['year'] = 2002
df_02_aug['month'] = 'aug'

flat_02_sep = {}
for name,arra in cyc_02_sep.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_02_sep[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_02_sep = pd.DataFrame(flat_02_sep)
df_02_sep['year'] = 2002
df_02_sep['month'] = 'sep'

flat_02_oct = {}
for name,arra in cyc_02_oct.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_02_oct[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_02_oct = pd.DataFrame(flat_02_oct)
df_02_oct['year'] = 2002
df_02_oct['month'] = 'oct'

flat_02_nov = {}
for name,arra in cyc_02_nov.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_02_nov[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_02_nov = pd.DataFrame(flat_02_nov)
df_02_nov['year'] = 2002
df_02_nov['month'] = 'nov'

flat_02_dec = {}
for name,arra in cyc_02_dec.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_02_dec[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_02_dec = pd.DataFrame(flat_02_dec)
df_02_dec['year'] = 2002
df_02_dec['month'] = 'dec'

print('saving dataframes')
df_02_jan.to_csv('\\df_02_jan.csv', index = False)
df_02_feb.to_csv('\\df_02_feb.csv', index = False)
df_02_mar.to_csv('\\df_02_mar.csv', index = False)
df_02_apr.to_csv('\\df_02_apr.csv', index = False)
df_02_may.to_csv('\\df_02_may.csv', index = False)
df_02_jun.to_csv('\\df_02_jun.csv', index = False)
df_02_jul.to_csv('\\df_02_jul.csv', index = False)
df_02_aug.to_csv('\\df_02_aug.csv', index = False)
df_02_sep.to_csv('\\df_02_sep.csv', index = False)
df_02_oct.to_csv('\\df_02_oct.csv', index = False)
df_02_nov.to_csv('\\df_02_nov.csv', index = False)
df_02_dec.to_csv('\\df_02_dec.csv', index = False)

In [ ]:
%%time

#2001 data

#data is shape (288, 441,741)
#first 0-23 in index[0] are the diurnal cycle for January of that 

filename_ERA = "mon_cycle_"

print('Doing Dictionaries')
#doing 2001
h = nc.Dataset(filename_ERA + '2001.nc')

cyc_01_jan = {}
cyc_01_jan['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][0:24]),1), mask = NA_mask_1deg_cyc ) 
cyc_01_jan['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_01_jan['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_01_jan['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_01_jan['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_01_jan['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_01_jan['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][0:24]),1), mask = NA_mask_1deg_cyc )
cyc_01_jan['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][0:24]),1), mask = NA_mask_1deg_cyc )


cyc_01_feb = {}
cyc_01_feb['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][24:48]),1), mask = NA_mask_1deg_cyc  )
cyc_01_feb['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_01_feb['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_01_feb['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_01_feb['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_01_feb['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_01_feb['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][24:48]),1), mask = NA_mask_1deg_cyc )
cyc_01_feb['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][24:48]),1), mask = NA_mask_1deg_cyc )


cyc_01_mar = {}
cyc_01_mar['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][48:72]),1), mask = NA_mask_1deg_cyc ) 
cyc_01_mar['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_01_mar['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_01_mar['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_01_mar['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_01_mar['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_01_mar['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][48:72]),1), mask = NA_mask_1deg_cyc )
cyc_01_mar['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][48:72]),1), mask = NA_mask_1deg_cyc )


cyc_01_apr = {}
cyc_01_apr['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][72:96]),1), mask = NA_mask_1deg_cyc ) 
cyc_01_apr['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_01_apr['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_01_apr['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_01_apr['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_01_apr['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_01_apr['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][72:96]),1), mask = NA_mask_1deg_cyc )
cyc_01_apr['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][72:96]),1), mask = NA_mask_1deg_cyc )


cyc_01_may = {}
cyc_01_may['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][96:120]),1), mask = NA_mask_1deg_cyc ) 
cyc_01_may['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_01_may['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_01_may['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_01_may['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_01_may['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_01_may['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][96:120]),1), mask = NA_mask_1deg_cyc )
cyc_01_may['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][96:120]),1), mask = NA_mask_1deg_cyc )


cyc_01_jun = {}
cyc_01_jun['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][120:144]),1), mask = NA_mask_1deg_cyc ) 
cyc_01_jun['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_01_jun['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_01_jun['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_01_jun['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_01_jun['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_01_jun['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][120:144]),1), mask = NA_mask_1deg_cyc )
cyc_01_jun['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][120:144]),1), mask = NA_mask_1deg_cyc )


cyc_01_jul = {}
cyc_01_jul['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][144:168]),1), mask = NA_mask_1deg_cyc ) 
cyc_01_jul['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_01_jul['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_01_jul['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_01_jul['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_01_jul['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_01_jul['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][144:168]),1), mask = NA_mask_1deg_cyc )
cyc_01_jul['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][144:168]),1), mask = NA_mask_1deg_cyc )


cyc_01_aug = {}
cyc_01_aug['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][168:192]),1), mask = NA_mask_1deg_cyc ) 
cyc_01_aug['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_01_aug['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_01_aug['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_01_aug['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_01_aug['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_01_aug['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][168:192]),1), mask = NA_mask_1deg_cyc )
cyc_01_aug['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][168:192]),1), mask = NA_mask_1deg_cyc )


cyc_01_sep = {}
cyc_01_sep['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][192:216]),1), mask = NA_mask_1deg_cyc ) 
cyc_01_sep['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_01_sep['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_01_sep['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_01_sep['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_01_sep['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_01_sep['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][192:216]),1), mask = NA_mask_1deg_cyc )
cyc_01_sep['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][192:216]),1), mask = NA_mask_1deg_cyc )


cyc_01_oct = {}
cyc_01_oct['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][216:240]),1), mask = NA_mask_1deg_cyc ) 
cyc_01_oct['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_01_oct['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_01_oct['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_01_oct['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_01_oct['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_01_oct['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][216:240]),1), mask = NA_mask_1deg_cyc )
cyc_01_oct['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][216:240]),1), mask = NA_mask_1deg_cyc )


cyc_01_nov = {}
cyc_01_nov['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][240:264]),1), mask = NA_mask_1deg_cyc ) 
cyc_01_nov['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_01_nov['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_01_nov['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_01_nov['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_01_nov['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_01_nov['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][240:264]),1), mask = NA_mask_1deg_cyc )
cyc_01_nov['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][240:264]),1), mask = NA_mask_1deg_cyc )


cyc_01_dec = {}
cyc_01_dec['2 metre dewpoint temperature'] = np.ma.array(downmean_3D((h.variables['d2m'][264:288]),1), mask = NA_mask_1deg_cyc ) 
cyc_01_dec['2 metre temperature'] = np.ma.array(downmean_3D((h.variables['t2m'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_01_dec['Surface latent heat flux'] = np.ma.array(downmean_3D((h.variables['slhf'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_01_dec['Surface net solar radiation'] = np.ma.array(downmean_3D((h.variables['ssr'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_01_dec['Surface net thermal radiation'] = np.ma.array(downmean_3D((h.variables['str'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_01_dec['Surface sensible heat flux'] = np.ma.array(downmean_3D((h.variables['sshf'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_01_dec['Surface pressure'] = np.ma.array(downmean_3D((h.variables['sp'][264:288]),1), mask = NA_mask_1deg_cyc )
cyc_01_dec['Total precipitation'] = np.ma.array(downmean_3D((h.variables['tp'][264:288]),1), mask = NA_mask_1deg_cyc )


h.close()


#flattening the arrays for each month and saving them as dfs
print('flattening arrays')

flat_01_jan = {}
for name,arra in cyc_01_jan.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_01_jan[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_01_jan = pd.DataFrame(flat_01_jan)
df_01_jan['year'] = 2001
df_01_jan['month'] = 'jan'

flat_01_feb = {}
for name,arra in cyc_01_feb.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_01_feb[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_01_feb = pd.DataFrame(flat_01_feb)
df_01_feb['year'] = 2001
df_01_feb['month'] = 'feb'

flat_01_mar = {}
for name,arra in cyc_01_mar.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_01_mar[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_01_mar = pd.DataFrame(flat_01_mar)
df_01_mar['year'] = 2001
df_01_mar['month'] = 'mar'

flat_01_apr = {}
for name,arra in cyc_01_apr.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_01_apr[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_01_apr = pd.DataFrame(flat_01_apr)
df_01_apr['year'] = 2001
df_01_apr['month'] = 'apr'

flat_01_may = {}
for name,arra in cyc_01_may.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_01_may[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_01_may = pd.DataFrame(flat_01_may)
df_01_may['year'] = 2001
df_01_may['month'] = 'may'

flat_01_jun = {}
for name,arra in cyc_01_jun.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_01_jun[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_01_jun = pd.DataFrame(flat_01_jun)
df_01_jun['year'] = 2001
df_01_jun['month'] = 'jun'

flat_01_jul = {}
for name,arra in cyc_01_jul.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_01_jul[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_01_jul = pd.DataFrame(flat_01_jul)
df_01_jul['year'] = 2001
df_01_jul['month'] = 'jul'

flat_01_aug = {}
for name,arra in cyc_01_aug.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_01_aug[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_01_aug = pd.DataFrame(flat_01_aug)
df_01_aug['year'] = 2001
df_01_aug['month'] = 'aug'

flat_01_sep = {}
for name,arra in cyc_01_sep.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_01_sep[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_01_sep = pd.DataFrame(flat_01_sep)
df_01_sep['year'] = 2001
df_01_sep['month'] = 'sep'

flat_01_oct = {}
for name,arra in cyc_01_oct.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_01_oct[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_01_oct = pd.DataFrame(flat_01_oct)
df_01_oct['year'] = 2001
df_01_oct['month'] = 'oct'

flat_01_nov = {}
for name,arra in cyc_01_nov.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_01_nov[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_01_nov = pd.DataFrame(flat_01_nov)
df_01_nov['year'] = 2001
df_01_nov['month'] = 'nov'

flat_01_dec = {}
for name,arra in cyc_01_dec.items():
#     print(name)
    temp_arr = arra.filled(np.nan).flatten() # flattening array
    flat_01_dec[name] = temp_arr[~np.isnan(temp_arr)] #removing nans

df_01_dec = pd.DataFrame(flat_01_dec)
df_01_dec['year'] = 2001
df_01_dec['month'] = 'dec'

print('saving dataframes')
df_01_jan.to_csv('\\df_01_jan.csv', index = False)
df_01_feb.to_csv('\\df_01_feb.csv', index = False)
df_01_mar.to_csv('\\df_01_mar.csv', index = False)
df_01_apr.to_csv('\\df_01_apr.csv', index = False)
df_01_may.to_csv('\\df_01_may.csv', index = False)
df_01_jun.to_csv('\\df_01_jun.csv', index = False)
df_01_jul.to_csv('\\df_01_jul.csv', index = False)
df_01_aug.to_csv('\\df_01_aug.csv', index = False)
df_01_sep.to_csv('\\df_01_sep.csv', index = False)
df_01_oct.to_csv('\\df_01_oct.csv', index = False)
df_01_nov.to_csv('\\df_01_nov.csv', index = False)
df_01_dec.to_csv('\\df_01_dec.csv', index = False)

This cell is extracting data from just June 2018, to be used to plot the values just for that mean monthly diurnal cycle. To get the entire cycle, the '00' will need to be replaced for every hour 00-23, and correspondingly changing the 0 in 'value[0]'  to 1-23 . 

In [ ]:
#this cell should be repeated for all hours of the day

# getting values for June 2018 00 a.m.
june_18_00a = {}

for key,value in cyc_18_jun.items():
    june_18_00a[key] = value[0].flatten(order = 'C')
    
jun_18_00a_df = pd.DataFrame({'RH' : june_18_00a['2 metre dewpoint temperature'],
                              'TA_ERA' : june_18_00a['2 metre temperature'],
                              'LE_F_MDS' : june_18_00a['Surface latent heat flux'],
                              'SW_IN_ERA' : june_18_00a['Surface net solar radiation'],
                              'LW_IN_ERA' : june_18_00a['Surface net thermal radiation'],
                              'H_F_MDS' : june_18_00a['Surface sensible heat flux'],
                              'PA_ERA' : june_18_00a['Surface pressure'],
                              'P_ERA' : june_18_00a['Total precipitation']
                             })

jun_18_00a_df.to_csv('june_2018_00am_ERA.csv', index = False)

